# Analyse de Canaux Multi-Échelles avec ZigZag et Lignes de Régression Pondérées

Ce notebook explore une approche d'analyse technique multi-échelles pour l'actif BTCUSDT (sur Binance), basée sur l'identification de pivots via l'indicateur ZigZag et la construction de canaux de tendance hiérarchiques (Macro, Méso, Micro).

**Objectifs :**

1.  **Extraire les points pivots significatifs** du prix horaire à l'aide d'une implémentation personnalisée de l'indicateur ZigZag.
2.  **Développer un algorithme** pour tracer des lignes de support et de résistance formant des canaux, en privilégiant les lignes qui "contiennent" strictement les pivots récents et en minimisant une erreur quadratique pondérée par le temps (`WSSE`).
3.  **Construire une hiérarchie de canaux** (Macro, Méso, Micro) où chaque niveau utilise les pivots définissant le niveau supérieur comme point de départ temporel.
4.  **Visualiser** les pivots, les canaux et leur évolution temporelle.
5.  **Explorer la sensibilité** de l'algorithme aux paramètres de pondération (`weight_power`, `wp`) et de fraction de pivots récents (`recent_pivot_fraction`, `rpf`).
6.  **(Optionnel / Futur)** Définir et backtester des stratégies de trading basées sur les interactions du prix avec ces canaux multi-échelles.

**Étapes Initiales :**

* Mise en place de l'environnement QuantConnect et téléchargement des données horaires BTCUSDT.
* Calcul d'une stratégie HODL simple comme référence de performance.
* Implémentation et visualisation de l'indicateur ZigZag.

## 1. Initialisation : Environnement et Données

Mise en place de l'environnement QuantConnect, définition de l'actif (BTCUSDT), de la période d'analyse et téléchargement des données horaires nécessaires.

In [32]:
# Import necessary libraries from the QC framework
from AlgorithmImports import *
from datetime import datetime
import pandas as pd

# Initialize QuantBook
qb = QuantBook()
qb.SetBrokerageModel(BrokerageName.Binance, AccountType.Cash)
# For this example, we use BTCUSDT and set it as our benchmark.
btc_symbol = qb.AddCrypto('BTCUSDT', Resolution.Hour, Market.Binance).Symbol
qb.SetBenchmark(btc_symbol)

# Define the backtesting period
start_date = datetime(2022, 1, 1)
end_date = datetime(2025, 4, 21)

# Download hourly history for BTCUSDT
bars_df = qb.History(btc_symbol, start_date, end_date, Resolution.Hour)
bars_df = bars_df.reset_index()

print('History loaded: ')
print(bars_df.head())



## 2. Stratégie de Référence : HODL

Calcul et visualisation de la performance d'une stratégie simple "Buy and Hold" (HODL) sur la période d'analyse. Ceci sert de benchmark pour évaluer toute stratégie de trading ultérieure.

In [33]:
# Assume we use the close prices from the hourly DataFrame as the HODL basis
initial_investment = 10000  # in USDT
first_price = bars_df['close'].iloc[0]
bars_df['HODL_Value'] = initial_investment * (bars_df['close'] / first_price)

# Print final portfolio value
final_value = bars_df['HODL_Value'].iloc[-1]
print(f"Final HODL portfolio value: {final_value:.2f} USDT")

# Plot HODL performance (optional: using matplotlib)
import matplotlib.pyplot as plt

# Disabling visualization temporarily for easier json copy

# plt.figure(figsize=(10, 5))
# plt.plot(bars_df['time'], bars_df['HODL_Value'], label='HODL Portfolio Value ')
# plt.xlabel('Time')
# plt.ylabel('Portfolio Value (USDT)')
# plt.title('HODL Strategy Performance')
# plt.legend()
# plt.show()


## 3. Détection des Pivots : Indicateur ZigZag

Cette section se concentre sur l'identification des points de retournement significatifs du marché à l'aide de l'indicateur ZigZag.

### 3.1. Fonction `classic_chart_zigzag`

Pour identifier les retournements de tendance significatifs, nous utilisons l'indicateur ZigZag. L'indicateur natif de QuantConnect pouvant présenter des comportements inattendus dans certains contextes de recherche, nous employons ici une fonction personnalisée `classic_chart_zigzag`.

**Fonctionnement :**

*   Elle identifie les points hauts (pivots High, type -1) et bas (pivots Low, type +1) sur les prix de clôture (`close`).
*   Un pivot est confirmé uniquement si le prix retrace d'un certain pourcentage (`thresholdPercent`) depuis le dernier extrême enregistré.
*   Dans cet exemple, `thresholdPercent=0.05` signifie qu'une baisse de 5% depuis le dernier plus haut est nécessaire pour confirmer ce plus haut comme un pivot High, et une hausse de 5% depuis le dernier plus bas est requise pour confirmer ce plus bas comme un pivot Low.
*   La fonction retourne une liste de pivots `(timestamp, price, type)`, garantissant une alternance stricte entre High et Low.

Cette liste de pivots (`pivotList`) servira de base à la construction de nos canaux.

In [34]:
import pandas as pd
import numpy as np

def classic_chart_zigzag(df, thresholdPercent=0.05):
    """
    Manually computes ZigZag pivots in the style of typical charting platforms.
    df must have columns 'time' and 'close'.
    thresholdPercent=0.05 => 5% retracement needed to lock in a pivot.
    Returns a list of pivots: (time, price, +1 or -1)
    +1 => Low pivot
    -1 => High pivot
    """
    if len(df) < 2:
        return []
    # We'll store (time, price, sign)
    pivots = []

    # 1) Start with the first bar as pivot
    firstBar = df.iloc[0]
    lastPivotPrice = firstBar['close']
    lastPivotTime  = firstBar['time']

    # 2) Determine initial direction (up or down) by comparing the second bar
    secondBar = df.iloc[1]
    directionUp = secondBar['close'] > lastPivotPrice  # True => up, False => down

    # If directionUp => we want to find new 'high extremes'
    # If directionUp=False => we want to find new 'low extremes'

    # Track the extreme in the current direction
    currentExtremePrice = lastPivotPrice
    currentExtremeTime  = lastPivotTime

    # We record the sign for the first pivot:
    #   if directionUp => first pivot is a Low (+1)
    #   else => first pivot is a High (-1)
    lastSign = +1 if directionUp else -1

    # We'll add that initial pivot to the list
    pivots.append((lastPivotTime, lastPivotPrice, lastSign))

    for i in range(1, len(df)):
        row = df.iloc[i]
        price = row['close']
        time  = row['time']

        # Update extremes in the current direction
        if directionUp:
            # If price is higher than currentExtreme => update extreme
            if price > currentExtremePrice:
                currentExtremePrice = price
                currentExtremeTime  = time
            else:
                # Check if we retraced enough to confirm a new pivot
                retrace = 1.0 - (price / currentExtremePrice)
                if retrace >= thresholdPercent:
                    # That means we lock in the old extreme as a High pivot
                    # Switch direction to down
                    pivots.append((currentExtremeTime, currentExtremePrice, -1))

                    # Now the new pivot is the current bar as 'low start'
                    directionUp = False
                    currentExtremePrice = price
                    currentExtremeTime  = time
        else:
            # directionDown => track new low extremes
            if price < currentExtremePrice:
                currentExtremePrice = price
                currentExtremeTime  = time
            else:
                # Check if we rallied enough to confirm a new pivot
                rally = (price / currentExtremePrice) - 1.0
                if rally >= thresholdPercent:
                    # Lock in the old extreme as a Low pivot
                    pivots.append((currentExtremeTime, currentExtremePrice, +1))

                    directionUp = True
                    currentExtremePrice = price
                    currentExtremeTime  = time

    # The last extreme can be appended as the final pivot if you want
    # e.g., treat the last bar as a pivot if you prefer
    finalPrice = currentExtremePrice
    finalTime  = currentExtremeTime
    finalSign  = +1 if not directionUp else -1  # Because if we ended going up, the last pivot is High
    pivots.append((finalTime, finalPrice, finalSign))

    # Now we have a strictly alternating pivot list
    return pivots

pivotList = classic_chart_zigzag(bars_df[['time','close']], thresholdPercent=0.05)


### 3.2. Visualisation du ZigZag

Nous affichons ici les prix de clôture horaires superposés avec l'indicateur ZigZag (lignes de connexion et marqueurs de pivots High/Low) pour vérifier visuellement la pertinence des points détectés.

In [35]:
def plot_manual_zigzag(bars_df, pivotList):
    import plotly.graph_objects as go

    # A) lines for raw price
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=bars_df['time'],
        y=bars_df['close'],
        mode='lines',
        name='Close Price',
        line=dict(color='blue')
    ))

    # B) build x and y arrays to connect pivot → pivot
    xline = []
    yline = []
    for (dt, price, _) in pivotList:
        xline.append(dt)
        yline.append(price)

    fig.add_trace(go.Scatter(
        x=xline, 
        y=yline,
        mode='lines',
        name='Classic ZigZag',
        line=dict(color='orange')
    ))

    # C) separate pivot sign for red or green markers
    high_x, high_y = [], []
    low_x,  low_y  = [], []

    for (dt, price, sign) in pivotList:
        if sign < 0:
            high_x.append(dt)
            high_y.append(price)
        else:
            low_x.append(dt)
            low_y.append(price)

    fig.add_trace(go.Scatter(
        x=high_x, 
        y=high_y,
        mode='markers',
        marker=dict(color='green', size=7),
        name='High Pivot'
    ))
    fig.add_trace(go.Scatter(
        x=low_x, 
        y=low_y,
        mode='markers',
        marker=dict(color='red', size=7),
        name='Low Pivot'
    ))

    fig.update_layout(
        title='BTCUSDT Hourly - Classic ZigZag (from scratch)',
        xaxis_title='Time',
        yaxis_title='Price (USDT)'
    )
    fig.show()

# THEN:

# plot_manual_zigzag(bars_df, pivotList)


### 3.3. Préparation des Données Pivots pour les Canaux

Avant de construire les canaux, une étape cruciale est de préparer les données des pivots extraits par le ZigZag :

1.  **Conversion en DataFrame :** Pour faciliter les manipulations.
2.  **Ajout de `time_numeric` :** Conversion du timestamp en secondes depuis l'epoch Unix. **Essentiel** pour les calculs géométriques (pente, ordonnée à l'origine) qui nécessitent une coordonnée 'x' numérique et linéaire.
3.  **Séparation High/Low :** Création de DataFrames distincts (`high_pivots`, `low_pivots`).
4.  **Stockage pour Vérification :** Création de copies NumPy (`all_high_pivots_for_check`, `all_low_pivots_for_check`) pour des vérifications rapides de contention lors de la recherche des meilleures lignes.

In [36]:
# Convert pivot list to DataFrame
pivots_df = pd.DataFrame(pivotList, columns=['time', 'price', 'type']) # type: +1 Low, -1 High

# Convert time to numerical representation (seconds since epoch) for accurate line calculations
# Note: Using time.timestamp() is generally good. Ensure your pandas version handles it correctly.
# Alternatively, convert to numpy datetime64 and then to integer/float.
pivots_df['time_numeric'] = pivots_df['time'].apply(lambda x: x.timestamp())

# Keep the original 'index' from the dataframe reset_index if needed for other purposes,
# but DO NOT use it for slope/channel calculations.
# pivots_df = pivots_df.reset_index() # Keep DataFrame index if needed

# Separate High and Low pivots
high_pivots = pivots_df[pivots_df['type'] == -1].copy()
low_pivots = pivots_df[pivots_df['type'] == +1].copy()

print("High Pivots with time_numeric:")
print(high_pivots.head())
print("\nLow Pivots with time_numeric:")
print(low_pivots.head())

# Store all pivots for global checks later
all_high_pivots_for_check = high_pivots[['time_numeric', 'price']].values
all_low_pivots_for_check = low_pivots[['time_numeric', 'price']].values

## 4. Construction des Canaux : Algorithme et Paramètres

Cette section détaille l'algorithme utilisé pour tracer les lignes de support et de résistance formant les canaux, ainsi que la configuration des paramètres qui influencent ce processus.


### 4.1. Algorithme Principal : `find_best_channel_line_strict_weighted`

La fonction `find_best_channel_line_strict_weighted` est au cœur de notre construction de canaux. Elle vise à identifier la "meilleure" ligne de support ou de résistance possible, définie par une paire de pivots (`p1`, `p2`), en se basant sur des critères stricts et une optimisation pondérée.

**Principes Clés :**

1.  **Contention Stricte (Zéro Violation) :** Une ligne candidate (support ou résistance) définie par `p1` et `p2` n'est considérée comme **valide** que si **tous** les autres pivots pertinents (tous les Lows pour un support, tous les Highs pour une résistance) se trouvent du "bon" côté de la ligne sur toute la période considérée. Si aucune ligne ne satisfait ce critère, la fonction retourne `None`.
2.  **Minimisation de l'Erreur Pondérée (WSSE) sur Fraction Récente :** Parmi toutes les lignes _strictement valides_, l'algorithme sélectionne celle qui minimise la **Somme Pondérée des Erreurs Quadratiques (Weighted Sum of Squared Errors - WSSE)**.
    *   **Pondération Temporelle (`weight_power`, `wp`) :** L'erreur de chaque pivot est pondérée par son ancienneté relative (`wp > 0` donne plus d'importance aux pivots récents).
    *   **Fraction Récente (`recent_pivot_fraction`, `rpf`) :** La WSSE n'est calculée que sur une fraction (`rpf`) des pivots les plus récents, focalisant l'optimisation sur la pertinence récente.
3.  **Paramétrisation :** `wp` et `rpf` sont ajustables indépendamment pour chaque type de ligne et échelle.

Cette approche garantit le respect de la structure globale des pivots tout en privilégiant la dynamique récente.

In [37]:
# CELLULE 13 : Définitions des Fonctions 

import numpy as np
import pandas as pd
import math 

# --- Helpers: get_line_params_time, check_point_position, calculate_weighted_sse ---
# (Copier ici les définitions complètes de ces 3 fonctions depuis la réponse précédente)
def get_line_params_time(p1_time_num, p1_price, p2_time_num, p2_price):
    time_diff = p2_time_num - p1_time_num
    if abs(time_diff) < 1e-9: return np.inf, p1_time_num
    m = (p2_price - p1_price) / time_diff
    c = p1_price - m * p1_time_num
    return m, c

def check_point_position(point_time_num, point_price, m, c, check_above, epsilon=1e-9):
    if m == np.inf: return False 
    line_y_at_point_x = m * point_time_num + c
    if check_above: return point_price >= line_y_at_point_x - epsilon
    else: return point_price <= line_y_at_point_x + epsilon

def calculate_weighted_sse(p1, p2, pivots_for_sse_np, time_min_wsse, time_max_wsse, weight_power=1.0):
    p1_time_num, p1_price = p1['time_numeric'], p1['price']
    p2_time_num, p2_price = p2['time_numeric'], p2['price']
    m, c = get_line_params_time(p1_time_num, p1_price, p2_time_num, p2_price)
    if m == np.inf: return np.inf
    total_wsse = 0.0; total_weight = 0.0; time_range = time_max_wsse - time_min_wsse
    if time_range < 1e-9: time_range = 1.0
    if len(pivots_for_sse_np) == 0: return 0.0
    for k in range(len(pivots_for_sse_np)):
        pk_time_num, pk_price = pivots_for_sse_np[k, 0], pivots_for_sse_np[k, 1]
        if abs(pk_time_num - p1_time_num) < 1e-9 or abs(pk_time_num - p2_time_num) < 1e-9: continue
        normalized_time = (pk_time_num - time_min_wsse) / time_range if time_range > 1e-9 else 0.5 
        weight = max(0, normalized_time) ** weight_power + 1e-9
        line_y_at_pk = m * pk_time_num + c
        error = pk_price - line_y_at_pk
        total_wsse += weight * (error**2)
        total_weight += weight
    return total_wsse / total_weight if total_weight > 1e-9 else 0.0

# --- Fonction Principale ---
def find_best_channel_line_strict_weighted(pivots_df, all_pivots_for_check_np, is_resistance,
                                           weight_power=1.0, recent_pivot_fraction=1.0):
    strictly_valid_lines_info = []
    n_pivots = len(pivots_df)
    if n_pivots < 2 or len(all_pivots_for_check_np) < 1: return None, None

    check_pivots_df = pd.DataFrame(all_pivots_for_check_np, columns=['time_numeric', 'price'])
    check_pivots_df = check_pivots_df.sort_values('time_numeric', ascending=True)
    n_total_check = len(check_pivots_df)
    safe_rpf = max(0.0, min(1.0, recent_pivot_fraction))
    n_keep_for_wsse = max(1, math.ceil(n_total_check * safe_rpf))
    recent_pivots_for_wsse_df = check_pivots_df.tail(n_keep_for_wsse).copy()
    
    if recent_pivots_for_wsse_df.empty or len(recent_pivots_for_wsse_df) < 1: return None, None 
    time_min_wsse = recent_pivots_for_wsse_df['time_numeric'].min()
    time_max_wsse = recent_pivots_for_wsse_df['time_numeric'].max()

    for i in range(n_pivots):
        p1 = pivots_df.iloc[i]
        for j in range(i + 1, n_pivots):
            p2 = pivots_df.iloc[j]
            if p1['time_numeric'] >= p2['time_numeric']: continue
            p1_time_num, p1_price = p1['time_numeric'], p1['price']
            p2_time_num, p2_price = p2['time_numeric'], p2['price']
            m, c = get_line_params_time(p1_time_num, p1_price, p2_time_num, p2_price)
            if m == np.inf: continue
            line_is_strictly_valid = True
            for k in range(n_total_check): 
                pk_time_num, pk_price = all_pivots_for_check_np[k, 0], all_pivots_for_check_np[k, 1]
                if abs(pk_time_num - p1_time_num) < 1e-9 or abs(pk_time_num - p2_time_num) < 1e-9: continue
                if not check_point_position(pk_time_num, pk_price, m, c, check_above=(not is_resistance)):
                    line_is_strictly_valid = False; break
            if line_is_strictly_valid:
                pivots_to_calc_wsse_on_df = recent_pivots_for_wsse_df[
                    (np.abs(recent_pivots_for_wsse_df['time_numeric'] - p1_time_num) > 1e-9) &
                    (np.abs(recent_pivots_for_wsse_df['time_numeric'] - p2_time_num) > 1e-9)
                ]
                if not pivots_to_calc_wsse_on_df.empty:
                    time_min_wsse_actual = pivots_to_calc_wsse_on_df['time_numeric'].min()
                    time_max_wsse_actual = pivots_to_calc_wsse_on_df['time_numeric'].max()
                    current_wsse = calculate_weighted_sse(p1, p2, pivots_to_calc_wsse_on_df[['time_numeric', 'price']].values,
                                                          time_min_wsse_actual, time_max_wsse_actual, weight_power)
                else: current_wsse = 0.0
                if current_wsse != np.inf:
                     strictly_valid_lines_info.append({ "p1": p1, "p2": p2, "wsse_recent": current_wsse })

    if not strictly_valid_lines_info: return None, None
    strictly_valid_lines_info.sort(key=lambda x: x['wsse_recent'])
    best_line_info = strictly_valid_lines_info[0]
    return best_line_info['p1'], best_line_info['p2']

### 4.2. Configuration des Paramètres de Canal

Ici, nous définissons les jeux de paramètres (`wp` et `rpf`) qui seront utilisés pour construire les canaux à différentes échelles.

1.  **`final_config` :** Configuration unique sélectionnée (potentiellement après optimisation) pour le tracé final, les snapshots et le backtesting éventuel. Contient `wp`/`rpf` pour Support/Résistance des niveaux Macro, Méso, Micro.
2.  **`configurations_to_explore` :** Liste de configurations générées pour tester la sensibilité de l'algorithme (ex: variation de `wp_micro_sup` et `rpf_micro_sup`). Les résultats alimenteront une visualisation comparative.

In [38]:
# CELLULE 14 : Définition des Configurations

import itertools


# --- Configurations à Explorer pour la Visualisation ---
print("\nGénération des configurations pour l'exploration...")
wp_ref = 2.0
rpf_ref = 1.0
# rpf_micro_sup_options = [1, 0.8, 0.66, 0.5, 0.3] 
rpf_micro_sup_options = [0.66, 0.5, 0.3] 
wp_micro_sup_options = [0.3, 0.5, 1.0, 2.0, 4.0, 8.0] 

configurations_to_explore = []
config_id_counter = 0
for rpf_micsup in rpf_micro_sup_options:
    for wp_micsup in wp_micro_sup_options:
        config_id_counter += 1
        config = {
            "label": f"Cfg{config_id_counter}_wpM={wp_micsup:.1f}_rpfM={rpf_micsup:.2f}",
            "wp_macro_res": wp_ref, "rpf_macro_res": rpf_ref,
            "wp_macro_sup": wp_ref, "rpf_macro_sup": rpf_ref,
            "wp_meso_res" : wp_ref, "rpf_meso_res" : rpf_ref,
            "wp_meso_sup" : wp_ref, "rpf_meso_sup" : rpf_ref,
            "wp_micro_res": wp_ref, "rpf_micro_res": rpf_ref,
            "wp_micro_sup": wp_micsup, "rpf_micro_sup": rpf_micsup,
        }
        configurations_to_explore.append(config)

print(f"Nombre de configurations à explorer: {len(configurations_to_explore)}")
results_list_explore = [] # Liste séparée pour les résultats de l'exploration

## 5. Calcul et Analyse des Canaux

Exécution des calculs pour déterminer les lignes de canal selon les configurations définies, suivie de l'analyse et de la visualisation des résultats.

### 5.1. Exploration des Paramètres (Calcul)

Cette cellule exécute le calcul des canaux pour **toutes** les configurations définies dans `configurations_to_explore`. Les pivots résultants pour chaque ligne de chaque configuration sont stockés dans `results_df_explore`. Ceci permet une analyse comparative ultérieure.

In [39]:
# CELLULE 15 : Calcul en Boucle pour l'Exploration (CORRIGÉE - Ajout time_numeric dans get_pivot_info)

import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm # S'assurer que tqdm est importé

print("Starting channel calculations for parameter exploration...")
if 'high_pivots' not in locals() or 'low_pivots' not in locals(): raise NameError("Pivots non définis.")
if 'find_best_channel_line_strict_weighted' not in locals(): raise NameError("Fonction de calcul non définie.")
if 'get_line_params_time' not in locals(): # S'assurer que la dépendance est là
    def get_line_params_time(p1_time_num, p1_price, p2_time_num, p2_price):
        time_diff = p2_time_num - p1_time_num
        if abs(time_diff) < 1e-9: return np.inf, p1_time_num
        m = (p2_price - p1_price) / time_diff
        c = p1_price - m * p1_time_num
        return m, c
if 'np' not in locals(): import numpy as np # Assurer import numpy
if 'math' not in locals(): import math # Assurer import math


results_list_explore = [] # S'assurer qu'elle est vide avant de commencer

# --- Définition de get_pivot_info DANS la cellule ---
# Correction: Ajouter time_numeric
def get_pivot_info(pivot):
    """Convertit une Series pivot en dictionnaire pour stockage, incluant time_numeric."""
    if pivot is None or not isinstance(pivot, pd.Series):
        return {'time': pd.NaT, 'price': np.nan, 'idx': -1, 'time_numeric': np.nan} # Valeurs par défaut
    # Récupérer 'idx' depuis le nom de la Series si disponible
    idx_val = pivot.name if hasattr(pivot, 'name') else -1
    # Assurer que time_numeric existe dans la Series source, le recalculer si besoin
    time_numeric_val = pivot.get('time_numeric', np.nan)
    if pd.isna(time_numeric_val) and not pd.isna(pivot.get('time')):
         try:
            # Convertir le timestamp pandas en objet datetime natif si nécessaire
            dt_obj = pivot['time']
            if isinstance(dt_obj, pd.Timestamp):
                 dt_obj = dt_obj.to_pydatetime()
            time_numeric_val = dt_obj.timestamp()
         except Exception as e:
             #print(f"WARN: Could not get timestamp for {pivot.get('time')}: {e}")
             time_numeric_val = np.nan # Fallback

    return {
        'time': pivot.get('time', pd.NaT),
        'price': pivot.get('price', np.nan),
        'idx': idx_val,
        'time_numeric': time_numeric_val # Utiliser la valeur récupérée/calculée
    }
# --- Fin définition get_pivot_info ---


for idx, config in enumerate(tqdm(configurations_to_explore, desc="Exploring Configs")): # Ajout tqdm ici
    # Stockage temporaire des résultats (tuples de Series ou (None, None))
    temp_channel_definitions = { "macro": {}, "meso": {}, "micro": {} }

    # --- Extraire params ---
    wp_macro_res=config['wp_macro_res']; rpf_macro_res=config['rpf_macro_res']
    wp_macro_sup=config['wp_macro_sup']; rpf_macro_sup=config['rpf_macro_sup']
    wp_meso_res =config['wp_meso_res'];  rpf_meso_res =config['rpf_meso_res']
    wp_meso_sup =config['wp_meso_sup'];  rpf_meso_sup =config['rpf_meso_sup']
    wp_micro_res=config['wp_micro_res']; rpf_micro_res=config['rpf_micro_res']
    wp_micro_sup=config['wp_micro_sup']; rpf_micro_sup=config['rpf_micro_sup']

    # --- Calcul Macro ---
    macro_success = False
    try:
        macro_res_series = find_best_channel_line_strict_weighted(high_pivots, all_high_pivots_for_check, True, wp_macro_res, rpf_macro_res)
        macro_sup_series = find_best_channel_line_strict_weighted(low_pivots, all_low_pivots_for_check, False, wp_macro_sup, rpf_macro_sup)
        temp_channel_definitions["macro"]["resistance"] = macro_res_series if macro_res_series else (None, None)
        temp_channel_definitions["macro"]["support"] = macro_sup_series if macro_sup_series else (None, None)
        if temp_channel_definitions["macro"]["resistance"][0] is not None and temp_channel_definitions["macro"]["support"][0] is not None:
            macro_success = True
    except Exception as e: print(f"ERROR Macro in {config['label']}: {e}")

    # --- Calcul Meso ---
    meso_success = False
    meso_start_time = None
    temp_channel_definitions["meso"]["resistance"] = (None, None)
    temp_channel_definitions["meso"]["support"] = (None, None)
    if macro_success:
        macro_pivots_list = [p for pair in temp_channel_definitions["macro"].values() for p in pair if p is not None]
        if len(macro_pivots_list) >= 2:
            try:
                meso_start_time = sorted([p['time'] for p in macro_pivots_list], reverse=True)[1]
                meso_high_f=high_pivots[high_pivots['time'] >= meso_start_time].copy(); meso_low_f=low_pivots[low_pivots['time'] >= meso_start_time].copy()
                if len(meso_high_f) >= 2 and len(meso_low_f) >= 2:
                    meso_high_np_check=meso_high_f[['time_numeric', 'price']].values; meso_low_np_check = meso_low_f[['time_numeric', 'price']].values
                    meso_res_series = find_best_channel_line_strict_weighted(meso_high_f, meso_high_np_check, True, wp_meso_res, rpf_meso_res)
                    meso_sup_series = find_best_channel_line_strict_weighted(meso_low_f, meso_low_np_check, False, wp_meso_sup, rpf_meso_sup)
                    temp_channel_definitions["meso"]["resistance"] = meso_res_series if meso_res_series else (None, None)
                    temp_channel_definitions["meso"]["support"] = meso_sup_series if meso_sup_series else (None, None)
                    if temp_channel_definitions["meso"]["resistance"][0] is not None and temp_channel_definitions["meso"]["support"][0] is not None:
                        meso_success = True
            except IndexError: pass
            except Exception as e: print(f"ERROR Meso in {config['label']}: {e}")

    # --- Calcul Micro ---
    micro_start_time = None
    temp_channel_definitions["micro"]["resistance"] = (None, None)
    temp_channel_definitions["micro"]["support"] = (None, None)
    if meso_success:
        meso_pivots_list = [p for pair in temp_channel_definitions["meso"].values() for p in pair if p is not None]
        if len(meso_pivots_list) >= 2:
            try:
                micro_start_time = sorted([p['time'] for p in meso_pivots_list], reverse=True)[1]
                micro_high_f=high_pivots[high_pivots['time'] >= micro_start_time].copy(); micro_low_f=low_pivots[low_pivots['time'] >= micro_start_time].copy()
                if len(micro_high_f) >= 2 and len(micro_low_f) >= 2:
                    micro_high_np_check=micro_high_f[['time_numeric', 'price']].values; micro_low_np_check=micro_low_f[['time_numeric', 'price']].values
                    micro_res_series = find_best_channel_line_strict_weighted(micro_high_f, micro_high_np_check, True, wp_micro_res, rpf_micro_res)
                    micro_sup_series = find_best_channel_line_strict_weighted(micro_low_f, micro_low_np_check, False, wp_micro_sup, rpf_micro_sup)
                    temp_channel_definitions["micro"]["resistance"] = micro_res_series if micro_res_series else (None, None)
                    temp_channel_definitions["micro"]["support"] = micro_sup_series if micro_sup_series else (None, None)
            except IndexError: pass
            except Exception as e: print(f"ERROR Micro in {config['label']}: {e}")

    # --- Stocker les résultats (UTILISER get_pivot_info ICI) ---
    result_data = {
        "config_index": idx, "config_label": config['label'], "params": config,
        "macro_res_p1": get_pivot_info(temp_channel_definitions['macro'].get('resistance', (None, None))[0]),
        "macro_res_p2": get_pivot_info(temp_channel_definitions['macro'].get('resistance', (None, None))[1]),
        "macro_sup_p1": get_pivot_info(temp_channel_definitions['macro'].get('support', (None, None))[0]),
        "macro_sup_p2": get_pivot_info(temp_channel_definitions['macro'].get('support', (None, None))[1]),
        "meso_res_p1": get_pivot_info(temp_channel_definitions['meso'].get('resistance', (None, None))[0]),
        "meso_res_p2": get_pivot_info(temp_channel_definitions['meso'].get('resistance', (None, None))[1]),
        "meso_sup_p1": get_pivot_info(temp_channel_definitions['meso'].get('support', (None, None))[0]),
        "meso_sup_p2": get_pivot_info(temp_channel_definitions['meso'].get('support', (None, None))[1]),
        "micro_res_p1": get_pivot_info(temp_channel_definitions['micro'].get('resistance', (None, None))[0]),
        "micro_res_p2": get_pivot_info(temp_channel_definitions['micro'].get('resistance', (None, None))[1]),
        "micro_sup_p1": get_pivot_info(temp_channel_definitions['micro'].get('support', (None, None))[0]),
        "micro_sup_p2": get_pivot_info(temp_channel_definitions['micro'].get('support', (None, None))[1]),
        "meso_start_t": meso_start_time if meso_start_time is not None else pd.NaT,
        "micro_start_t": micro_start_time if micro_start_time is not None else pd.NaT,
    }
    results_list_explore.append(result_data)
    # print(f"--> Result Micro Support: p1={result_data['micro_sup_p1'].get('time')} - p2={result_data['micro_sup_p2'].get('time')}") # Moins verbeux

print("\n--- Channel exploration calculations finished ---")
results_df_explore = pd.DataFrame(results_list_explore)
print("\n--- Exploration Results Summary (Micro Support Pivots) ---")
if not results_df_explore.empty:
    if 'micro_sup_p1' in results_df_explore.columns and 'micro_sup_p2' in results_df_explore.columns:
        pd.set_option('display.max_rows', None); pd.set_option('display.max_columns', None); pd.set_option('display.width', 2000)
        results_df_explore['mic_s1_t'] = results_df_explore['micro_sup_p1'].apply(lambda x: x.get('time') if isinstance(x, dict) else pd.NaT)
        results_df_explore['mic_s2_t'] = results_df_explore['micro_sup_p2'].apply(lambda x: x.get('time') if isinstance(x, dict) else pd.NaT)
        print(results_df_explore[['config_index', 'config_label', 'mic_s1_t', 'mic_s2_t']].to_string())
        pd.reset_option('display.max_rows'); pd.reset_option('display.max_columns'); pd.reset_option('display.width')
    else:
        print("WARN: Colonnes 'micro_sup_p1' ou 'micro_sup_p2' manquantes dans results_df_explore.")
        print(results_df_explore.head())
else: print("results_df_explore is empty.")

### 5.2. Identification de la Configuration Cible (`final_config`)

Examen des résultats de l'exploration (`results_df_explore`), potentiellement basé sur des critères visuels ou quantitatifs (non implémentés ici), pour sélectionner la configuration (`final_config`) qui semble la plus pertinente pour une analyse plus approfondie (visualisation finale, snapshots, backtesting).

In [40]:
# CELLULE 16 : Identification Configuration Cible

target_p1_date = pd.Timestamp("2025-04-07 07:00:00")
target_p2_date = pd.Timestamp("2025-04-09 04:00:00")
tolerance = pd.Timedelta(hours=6) 
best_config_index = -1 # Default

if 'results_df_explore' in locals() and not results_df_explore.empty:
    # Convertir les colonnes de temps si elles ne sont pas déjà datetime (elles devraient l'être)
    results_df_explore['mic_s1_t_dt'] = pd.to_datetime(results_df_explore['micro_sup_p1'].apply(lambda x: x['time']))
    results_df_explore['mic_s2_t_dt'] = pd.to_datetime(results_df_explore['micro_sup_p2'].apply(lambda x: x['time']))
    
    target_configs_df = results_df_explore[
        (results_df_explore['mic_s1_t_dt'] >= target_p1_date - tolerance) & (results_df_explore['mic_s1_t_dt'] <= target_p1_date + tolerance) &
        (results_df_explore['mic_s2_t_dt'] >= target_p2_date - tolerance) & (results_df_explore['mic_s2_t_dt'] <= target_p2_date + tolerance)
    ]
    
    if not target_configs_df.empty:
        print(f"\n--- Configurations Cible Trouvées ---")
        print(target_configs_df[['config_index', 'config_label', 'mic_s1_t', 'mic_s2_t']].to_string())
        best_config_index = target_configs_df.iloc[0]['config_index']
        print(f"\n==> Sélection auto de la 1ère config cible: Index {best_config_index} ({target_configs_df.iloc[0]['config_label']})\n")
        # Stocker la meilleure configuration trouvée
        final_config = configurations_to_explore[best_config_index]
    else:
        print(f"\n--- ATTENTION : Aucune config trouvée pour Support Micro cible ({target_p1_date} - {target_p2_date}) ---")
        # Fallback: Utiliser la config par défaut ou la dernière ? Utilisons la config par défaut définie plus haut.
        best_config_index = -99 # Marqueur spécial
        final_config = { "label": "Default_Config", # Fallback
            "wp_macro_res": 2.0, "rpf_macro_res": 1.0, "wp_macro_sup": 2.0, "rpf_macro_sup": 1.0,
            "wp_meso_res" : 2.0, "rpf_meso_res" : 1.0, "wp_meso_sup" : 2.0, "rpf_meso_sup" : 1.0,
            "wp_micro_res": 2.0, "rpf_micro_res": 1.0, "wp_micro_sup": 2.0, "rpf_micro_sup": 1.0, }
        print(f"WARN: Utilisation de la configuration par défaut: {final_config['label']}")
        
else:
    print("results_df_explore non trouvé ou vide. Utilisation d'une config par défaut.")
    best_config_index = -99
    final_config = { "label": "Default_Config", # Fallback
            "wp_macro_res": 2.0, "rpf_macro_res": 1.0, "wp_macro_sup": 2.0, "rpf_macro_sup": 1.0,
            # ... (autres params par défaut) ...
             }

# --- Calcul Final avec la Meilleure Configuration ---
# (Optionnel ici, on peut le faire juste avant le plot 2D final si on veut séparer)
print("Recalculating channels with the selected final configuration...")
final_channel_definitions = {"macro": {}, "meso": {}, "micro": {}}
# ... (Copier/Coller la logique de calcul complète pour UNE config, utilisant final_config) ...
# (Identique à la structure interne de la boucle dans la Cellule 15, mais sans boucle)
# --- Macro ---
try:
    macro_res = find_best_channel_line_strict_weighted(high_pivots, all_high_pivots_for_check, True, final_config['wp_macro_res'], final_config['rpf_macro_res'])
    macro_sup = find_best_channel_line_strict_weighted(low_pivots, all_low_pivots_for_check, False, final_config['wp_macro_sup'], final_config['rpf_macro_sup'])
    final_channel_definitions["macro"]["resistance"] = macro_res if macro_res else (None, None)
    final_channel_definitions["macro"]["support"] = macro_sup if macro_sup else (None, None)
except Exception as e: print(f"ERROR Final Macro: {e}")
# --- Meso ---
meso_start_time = None; macro_pivots_list = [p for pair in final_channel_definitions["macro"].values() if pair and pair[0] is not None and pair[1] is not None for p in pair]
if len(macro_pivots_list) >= 2:
    try:
        meso_start_time = sorted([p['time'] for p in macro_pivots_list], reverse=True)[1]
        meso_high_f=high_pivots[high_pivots['time'] >= meso_start_time].copy(); meso_low_f=low_pivots[low_pivots['time'] >= meso_start_time].copy()
        meso_high_np=meso_high_f[['time_numeric', 'price']].values; meso_low_np = meso_low_f[['time_numeric', 'price']].values
        if len(meso_high_f) >= 2 and len(meso_low_f) >= 2:
             meso_res = find_best_channel_line_strict_weighted(meso_high_f, meso_high_np, True, final_config['wp_meso_res'], final_config['rpf_meso_res'])
             meso_sup = find_best_channel_line_strict_weighted(meso_low_f, meso_low_np, False, final_config['wp_meso_sup'], final_config['rpf_meso_sup'])
             final_channel_definitions["meso"]["resistance"] = meso_res if meso_res else (None, None)
             final_channel_definitions["meso"]["support"] = meso_sup if meso_sup else (None, None)
    except Exception as e: print(f"ERROR Final Meso: {e}")
# --- Micro ---
micro_start_time = None; meso_pivots_list = [p for pair in final_channel_definitions["meso"].values() if pair and pair[0] is not None and pair[1] is not None for p in pair]
if len(meso_pivots_list) >= 2:
    try:
        micro_start_time = sorted([p['time'] for p in meso_pivots_list], reverse=True)[1]
        micro_high_f=high_pivots[high_pivots['time'] >= micro_start_time].copy(); micro_low_f=low_pivots[low_pivots['time'] >= micro_start_time].copy()
        micro_high_np=micro_high_f[['time_numeric', 'price']].values; micro_low_np=micro_low_f[['time_numeric', 'price']].values
        if len(micro_high_f) >= 2 and len(micro_low_f) >= 2:
             micro_res = find_best_channel_line_strict_weighted(micro_high_f, micro_high_np, True, final_config['wp_micro_res'], final_config['rpf_micro_res'])
             micro_sup = find_best_channel_line_strict_weighted(micro_low_f, micro_low_np, False, final_config['wp_micro_sup'], final_config['rpf_micro_sup'])
             final_channel_definitions["micro"]["resistance"] = micro_res if micro_res else (None, None)
             final_channel_definitions["micro"]["support"] = micro_sup if micro_sup else (None, None)
    except Exception as e: print(f"ERROR Final Micro: {e}")

print("\nFinal channels calculated using selected configuration.")

### 5.3. Visualisation Finale (Configuration Cible)

Cette section utilise Plotly pour générer une visualisation complète intégrant :

*   Le **prix** de clôture horaire.
*   Les **pivots** ZigZag (High/Low).
*   Les **lignes de connexion** du ZigZag.
*   Les **canaux Macro, Méso et Micro** (support et résistance) calculés en utilisant la configuration sélectionnée (`final_config`).

**Caractéristiques du Tracé :**

*   Niveaux de canal distincts par couleur/style.
*   Lignes étendues jusqu'à la fin des données.
*   Pivots définissant chaque ligne mis en évidence (marqueurs étoiles).

In [41]:
import plotly.graph_objects as go

def plot_channels_and_zigzag(bars_df, pivotList, channel_definitions):
    
    fig = go.Figure()

    # A) Plot raw price
    fig.add_trace(go.Scatter(
        x=bars_df['time'],
        y=bars_df['close'],
        mode='lines',
        name='Close Price',
        line=dict(color='rgba(100, 100, 200, 0.8)') # Light blue
    ))

    # B) Plot ZigZag lines connecting pivots
    xline = [p[0] for p in pivotList]
    yline = [p[1] for p in pivotList]
    fig.add_trace(go.Scatter(
        x=xline, 
        y=yline,
        mode='lines',
        name='ZigZag Line',
        line=dict(color='orange', dash='dot')
    ))

    # C) Plot High/Low pivot markers
    high_x = [p[0] for p in pivotList if p[2] < 0]
    high_y = [p[1] for p in pivotList if p[2] < 0]
    low_x  = [p[0] for p in pivotList if p[2] > 0]
    low_y  = [p[1] for p in pivotList if p[2] > 0]

    fig.add_trace(go.Scatter(
        x=high_x, y=high_y, mode='markers',
        marker=dict(color='red', size=7, symbol='diamond'), name='High Pivot'
    ))
    fig.add_trace(go.Scatter(
        x=low_x, y=low_y, mode='markers',
        marker=dict(color='green', size=7, symbol='circle'), name='Low Pivot'
    ))

    # D) Plot Channel Lines (Revised for Extension)
    channel_colors = {
        "macro": "rgba(60, 60, 60, 0.8)", # Dark Grey
        "meso": "rgba(0, 0, 255, 0.6)",   # Blue
        "micro": "rgba(255, 0, 255, 0.7)" # Magenta
    }
    channel_styles = {
        "macro": "solid",
        "meso": "dash",
        "micro": "dot"
    }
    # Get the overall time range for extending lines
    plot_start_time = bars_df['time'].iloc[0]
    plot_end_time = bars_df['time'].iloc[-1]
    plot_start_time_num = plot_start_time.timestamp()
    plot_end_time_num = plot_end_time.timestamp()
    for name, definition in channel_definitions.items():
        color = channel_colors[name]
        style = channel_styles[name]
        for line_type in ["resistance", "support"]:
            p1, p2 = definition[line_type]
            if p1 is not None and p2 is not None:
                # Calculate line parameters using time_numeric
                m, c = get_line_params_time(p1['time_numeric'], p1['price'], p2['time_numeric'], p2['price'])
                # Calculate line extension points for plotting
                # Start from the time of the first defining pivot
                line_plot_start_time = p1['time']
                line_plot_start_time_num = p1['time_numeric']
                line_plot_start_y = p1['price'] # More accurately: m * line_plot_start_time_num + c
                # End at the time of the last data point in the plot
                line_plot_end_time = plot_end_time
                line_plot_end_time_num = plot_end_time_num
                line_plot_end_y = m * line_plot_end_time_num + c
                # Add the extended line trace
                fig.add_trace(go.Scatter(
                    x=[line_plot_start_time, line_plot_end_time],
                    y=[line_plot_start_y, line_plot_end_y],
                    mode='lines',
                    name=f'{name.capitalize()} {line_type.capitalize()}',
                    line=dict(color=color, width=2, dash=style)
                ))
                # Highlight the defining pivots
                fig.add_trace(go.Scatter(
                    x=[p1['time'], p2['time']],
                    y=[p1['price'], p2['price']],
                    mode='markers',
                    marker=dict(color=color, size=9,
                                symbol='star' if line_type == 'resistance' else 'star-open'),
                    showlegend=False
                ))
    fig.update_layout(
        title="BTCUSDT Hourly - ZigZag Pivots with Macro/Meso/Micro Channels (Hierarchical Envelope)",
        xaxis_title="Time",
        yaxis_title="Price (USDT)",
        xaxis_rangeslider_visible=False
    )
    fig.show()


# --- Exécuter le Plotting avec la configuration FINALE ---

# S'assurer que final_channel_definitions existe et contient les bons canaux
if 'final_channel_definitions' in locals():
    plot_channels_and_zigzag(bars_df, pivotList, final_channel_definitions)
    print("Plotting avec la configuration finale terminée.")
else:
    print("ERREUR: `final_channel_definitions` non trouvées. Exécuter la cellule 16 pour recalculer les canaux finaux.")


### 5.4. Visualisation Comparative (Exploration Paramètres)

Ce graphique utilise une grille de sous-graphiques (subplots) pour visualiser comment les canaux **Macro, Méso et Micro** varient en fonction des **différentes configurations testées** dans `configurations_to_explore`.

**Organisation de la Grille :**

*   Chaque sous-graphique correspond à une configuration testée.
*   Le titre indique les paramètres variables (ex: `wp_micro_sup`, `rpf_micro_sup`).
*   Chaque sous-graphique **zoome automatiquement** sur une fenêtre temporelle (X) et une plage de prix (Y) déterminées par les **pivots définissant le canal Méso** de cette configuration spécifique, pour mieux voir l'impact local des paramètres.
*   Tous les canaux (Macro, Méso, Micro) et pivots ZigZag pertinents sont affichés dans chaque sous-graphique zoomé.

Aide à comprendre la **stabilité** et la **sensibilité** de l'algorithme aux variations des paramètres `wp` et `rpf`.

In [42]:
# CELLULE 28 : Visualisation 2D (Grille 4x4) Exploration - Tous Canaux, Zoom X/Y sur Pivots Meso par Config

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import math

# --- Pré-requis ---
if 'results_df_explore' not in locals() or results_df_explore.empty:
    print("ERREUR: results_df_explore non disponible pour la visualisation.")
elif 'get_line_params_time' not in locals():
    print("ERREUR: La fonction get_line_params_time n'est pas définie.")
elif 'bars_df' not in locals():
     print("ERREUR: `bars_df` non disponible pour tracer le prix.")
elif 'pivotList' not in locals():
     print("ERREUR: `pivotList` global non disponible pour tracer les pivots.")
else:
    # --- MODIFICATION : Sélectionner N premières configurations (16 pour une grille 4x4) ---
    n_configs_to_plot = 16 # <--- Changé de 9 à 16
    configs_to_plot = results_df_explore.head(n_configs_to_plot).copy()
    if len(configs_to_plot) < 1: # Vérifier s'il y a au moins une config
         print("ERREUR: Aucune configuration à plotter.")
         configs_to_plot = None # Marqueur pour ne pas continuer
    elif len(configs_to_plot) < n_configs_to_plot:
        print(f"WARN: Moins de {n_configs_to_plot} configurations disponibles ({len(configs_to_plot)}). Affichage d'une grille partielle.")
        n_configs_to_plot = len(configs_to_plot) # Ajuster le nombre réel

if 'configs_to_plot' in locals() and configs_to_plot is not None:
    cols = 4 # <--- Changé de 3 à 4
    rows = math.ceil(n_configs_to_plot / cols)
    subplot_titles = [row['config_label'] for index, row in configs_to_plot.iterrows()]

    # S'assurer qu'il y a assez de titres pour la grille (remplir si besoin)
    subplot_titles.extend([''] * (rows * cols - len(subplot_titles)))

    fig = make_subplots(rows=rows, cols=cols, subplot_titles=subplot_titles,
                        shared_xaxes=False, # Axe X indépendant
                        shared_yaxes=False, # Axe Y indépendant
                        vertical_spacing=0.06, horizontal_spacing=0.04) # Espacement ajusté

    channel_colors = {"macro": "rgba(60, 60, 60, 0.8)", "meso": "rgba(0, 0, 255, 0.6)", "micro": "rgba(255, 0, 255, 0.7)"}
    channel_styles = {"macro": "solid", "meso": "dash", "micro": "dot"}

    # Pivots globaux (ceux calculés sur tout l'historique initialement)
    high_x_all = [p[0] for p in pivotList if p[2] < 0]
    high_y_all = [p[1] for p in pivotList if p[2] < 0]
    low_x_all  = [p[0] for p in pivotList if p[2] > 0]
    low_y_all  = [p[1] for p in pivotList if p[2] > 0]

    # --- Itérer sur les configurations SÉLECTIONNÉES ---
    for plot_index, (index, row_data) in enumerate(configs_to_plot.iterrows()):
        row_idx = (plot_index // cols) + 1
        col_idx = (plot_index % cols) + 1

        config_label = row_data['config_label']

        # 1. Récupérer les canaux pour CETTE configuration
        channels_for_this_config = {
            "macro": {"resistance": (row_data["macro_res_p1"], row_data["macro_res_p2"]), "support": (row_data["macro_sup_p1"], row_data["macro_sup_p2"])},
            "meso": {"resistance": (row_data["meso_res_p1"], row_data["meso_res_p2"]), "support": (row_data["meso_sup_p1"], row_data["meso_sup_p2"])},
            "micro": {"resistance": (row_data["micro_res_p1"], row_data["micro_res_p2"]), "support": (row_data["micro_sup_p1"], row_data["micro_sup_p2"])}
        }

        # 2. Calculer la plage X et Y basée sur les pivots Meso de CETTE config
        meso_pivots_data = []
        yaxis_range_subplot = None
        xaxis_range_subplot = None

        meso_def = channels_for_this_config.get("meso", {})
        meso_res_p1_info, meso_res_p2_info = meso_def.get("resistance", ({},{}))
        meso_sup_p1_info, meso_sup_p2_info = meso_def.get("support", ({},{}))

        for p_info in [meso_res_p1_info, meso_res_p2_info, meso_sup_p1_info, meso_sup_p2_info]:
            if isinstance(p_info, dict) and not pd.isna(p_info.get('time')) and not pd.isna(p_info.get('price')):
                meso_pivots_data.append({'time': pd.to_datetime(p_info['time']), 'price': p_info['price']})

        if len(meso_pivots_data) >= 2:
            meso_pivots_df = pd.DataFrame(meso_pivots_data)
            min_p_meso = meso_pivots_df['price'].min()
            max_p_meso = meso_pivots_df['price'].max()
            min_t_meso = meso_pivots_df['time'].min()
            max_t_meso = meso_pivots_df['time'].max()

            # Calcul Plage Y
            height_meso = max_p_meso - min_p_meso
            margin_y = max(height_meso * 0.30, (min_p_meso + max_p_meso)/2 * 0.05)
            yaxis_range_subplot = [min_p_meso - margin_y, max_p_meso + margin_y]

            # Calcul Plage X
            duration_meso = max_t_meso - min_t_meso if max_t_meso > min_t_meso else pd.Timedelta(days=1)
            margin_t_before = max(duration_meso * 0.5, pd.Timedelta(days=60))
            margin_t_after = max(duration_meso * 0.2, pd.Timedelta(days=30))
            xaxis_range_subplot = [min_t_meso - margin_t_before, max_t_meso + margin_t_after]
        else:
            # Fallback si pas de pivots meso trouvés pour cette config
            if not bars_df.empty:
                fallback_end_time = bars_df['time'].iloc[-1]
                fallback_start_time = fallback_end_time - pd.Timedelta(days=180) # Lookback par défaut
                xaxis_range_subplot = [fallback_start_time, fallback_end_time]


        # 3. Filtrer les données à afficher dans cette plage temporelle
        if xaxis_range_subplot:
            bars_subplot = bars_df[(bars_df['time'] >= xaxis_range_subplot[0]) & (bars_df['time'] <= xaxis_range_subplot[1])].copy()
            high_x_plot = [t for t in high_x_all if t >= xaxis_range_subplot[0] and t <= xaxis_range_subplot[1]]
            high_y_plot = [high_y_all[i] for i, t in enumerate(high_x_all) if t >= xaxis_range_subplot[0] and t <= xaxis_range_subplot[1]]
            low_x_plot = [t for t in low_x_all if t >= xaxis_range_subplot[0] and t <= xaxis_range_subplot[1]]
            low_y_plot = [low_y_all[i] for i, t in enumerate(low_x_all) if t >= xaxis_range_subplot[0] and t <= xaxis_range_subplot[1]]
        else: # Si pas de plage X définie (ne devrait pas arriver avec le fallback)
            bars_subplot = bars_df.copy()
            high_x_plot, high_y_plot = high_x_all, high_y_all
            low_x_plot, low_y_plot = low_x_all, low_y_all
            if not bars_subplot.empty:
                 xaxis_range_subplot = [bars_subplot['time'].min(), bars_subplot['time'].max()] # Fallback range X

        # Fallback pour Y si toujours None
        if yaxis_range_subplot is None and not bars_subplot.empty:
            min_p = bars_subplot['low'].min()
            max_p = bars_subplot['high'].max()
            margin_y = (max_p - min_p) * 0.1
            yaxis_range_subplot = [min_p - margin_y, max_p + margin_y]

        # 4. Tracer le prix (section filtrée)
        if not bars_subplot.empty:
            fig.add_trace(go.Scatter(x=bars_subplot['time'], y=bars_subplot['close'], mode='lines',
                                     line=dict(color='rgba(150, 150, 150, 0.5)'), name='Close',
                                     showlegend=False),
                          row=row_idx, col=col_idx)

        # 5. Tracer les pivots globaux (section filtrée)
        fig.add_trace(go.Scatter(x=high_x_plot, y=high_y_plot, mode='markers', name='High Pivot',
                                 marker=dict(color='red', size=4, symbol='diamond-open'), showlegend=(plot_index==0)), # Légende 1 fois
                      row=row_idx, col=col_idx)
        fig.add_trace(go.Scatter(x=low_x_plot, y=low_y_plot, mode='markers', name='Low Pivot',
                                 marker=dict(color='green', size=4, symbol='circle-open'), showlegend=(plot_index==0)), # Légende 1 fois
                      row=row_idx, col=col_idx)


        # 6. Tracer les 3 canaux pour CETTE configuration
        if not bars_subplot.empty:
            plot_start_time_dt_eff = bars_subplot['time'].min()
            plot_end_time_dt_eff = bars_subplot['time'].max()
            plot_start_time_num_eff = plot_start_time_dt_eff.timestamp()
            plot_end_time_num_eff = plot_end_time_dt_eff.timestamp()

            for channel_name, definition in channels_for_this_config.items():
                for line_type in ["resistance", "support"]:
                    p1_info, p2_info = definition.get(line_type, ({}, {}))

                    if (isinstance(p1_info, dict) and isinstance(p2_info, dict) and
                        not pd.isna(p1_info.get('time')) and not pd.isna(p2_info.get('time')) and
                        not pd.isna(p1_info.get('time_numeric')) and not pd.isna(p2_info.get('time_numeric')) and
                        not pd.isna(p1_info.get('price')) and not pd.isna(p2_info.get('price'))):

                        p1_time_num = p1_info['time_numeric']
                        p2_time_num = p2_info['time_numeric']
                        p1_price = p1_info['price']
                        p2_price = p2_info['price']
                        p1_time = pd.to_datetime(p1_info['time'])
                        p2_time = pd.to_datetime(p2_info['time'])

                        try:
                            m, c = get_line_params_time(p1_time_num, p1_price, p2_time_num, p2_price)
                            if m != np.inf:
                                # Calculer les points Y aux extrémités de la FENETRE VISIBLE
                                line_start_y_plot = m * plot_start_time_num_eff + c
                                line_end_y_plot = m * plot_end_time_num_eff + c

                                fig.add_trace(go.Scatter(
                                    x=[plot_start_time_dt_eff, plot_end_time_dt_eff], y=[line_start_y_plot, line_end_y_plot],
                                    mode='lines', name=f"{channel_name.capitalize()} {line_type.capitalize()}",
                                    line=dict(color=channel_colors[channel_name], width=1.5, dash=channel_styles[channel_name]),
                                    legendgroup=f"{channel_name}_{line_type}",
                                    showlegend=(plot_index==0) # Légende juste pour le premier subplot
                                ), row=row_idx, col=col_idx)

                                # Marquer les pivots utilisés (plus gros) s'ils sont visibles
                                pivots_x_to_mark = []
                                pivots_y_to_mark = []
                                if xaxis_range_subplot and p1_time >= xaxis_range_subplot[0] and p1_time <= xaxis_range_subplot[1]:
                                    pivots_x_to_mark.append(p1_time)
                                    pivots_y_to_mark.append(p1_price)
                                if xaxis_range_subplot and p2_time >= xaxis_range_subplot[0] and p2_time <= xaxis_range_subplot[1]:
                                    pivots_x_to_mark.append(p2_time)
                                    pivots_y_to_mark.append(p2_price)

                                if pivots_x_to_mark:
                                    fig.add_trace(go.Scatter(
                                        x=pivots_x_to_mark, y=pivots_y_to_mark,
                                        mode='markers', marker=dict(color=channel_colors[channel_name], size=7,
                                                                      symbol='star' if line_type == 'resistance' else 'star-open'),
                                        showlegend=False, hoverinfo='skip'
                                    ), row=row_idx, col=col_idx)
                        except Exception as e: print(f"Error plotting line {channel_name} {line_type} for {config_label}: {e}")

        # Appliquer les zooms X et Y calculés pour ce subplot
        if yaxis_range_subplot: fig.update_yaxes(range=yaxis_range_subplot, row=row_idx, col=col_idx)
        if xaxis_range_subplot: fig.update_xaxes(range=xaxis_range_subplot, row=row_idx, col=col_idx)

        # Masquer les ticks X/Y inutiles
        if row_idx < rows: fig.update_xaxes(showticklabels=False, row=row_idx, col=col_idx)
        if col_idx > 1: fig.update_yaxes(showticklabels=False, row=row_idx, col=col_idx)


    # --- Configurer Layout Final ---
    fig.update_layout(
        title=f'Exploration Paramètres ({n_configs_to_plot} Configs): Canaux & Zoom X/Y sur Pivots Meso',
        height=250 * rows + 80, # Ajuster hauteur pour 4 lignes
        width=1200, # Un peu plus large pour 4 colonnes
        hovermode='x unified',
        legend=dict(orientation="h", yanchor="bottom", y=-0.05, xanchor="center", x=0.5) # Légende un peu plus bas
    )
    # Mettre à jour la taille des titres des subplots
    for i, annot in enumerate(fig.layout.annotations):
        if i < n_configs_to_plot: # Seulement pour les subplots utilisés
            annot.update(font=dict(size=9))

    fig.show()

## 6. Analyse Temporelle : Snapshots Historiques

Pour évaluer la robustesse et l'évolution des canaux dans le temps, nous recalculons l'ensemble de la structure de canaux (Macro, Méso, Micro) tels qu'ils auraient été tracés à différentes **dates passées ("snapshots")**, en utilisant la **configuration fixe** `final_config`.

### 6.1. Calcul des Snapshots

**Processus :**

1.  **Dates de Snapshot :** Une liste de dates historiques est définie (ex: fin de mois).
2.  **Configuration Fixe :** `final_config` est utilisée pour tous les calculs.
3.  **Calcul Itératif :** Pour chaque date de snapshot :
    *   L'historique de prix *jusqu'à cette date* est utilisé.
    *   Les pivots ZigZag sont recalculés sur cet historique tronqué.
    *   Les canaux Macro, Méso et Micro sont entièrement recalculés (avec la logique de **fallback** si nécessaire pour Meso/Micro) en utilisant les paramètres de `final_config`.
4.  **Stockage :** Les pivots et définitions de canaux résultants (`snapshot_results`) sont stockés pour chaque date.

Permet de vérifier si les canaux récents étaient déjà présents ou similaires dans le passé.

In [43]:
# CELLULE HELPER (MODIFIÉE) : Fonction pour Calculer Tous les Canaux à une Date Donnée AVEC FALLBACK

import pandas as pd
import numpy as np
import math
import traceback # Pour le débogage si besoin

# Assurer que les dépendances sont chargées (elles devraient l'être par les cellules précédentes)
if 'classic_chart_zigzag' not in locals(): raise NameError("classic_chart_zigzag non définie")
if 'find_best_channel_line_strict_weighted' not in locals(): raise NameError("find_best_channel_line_strict_weighted non définie")
if 'get_pivot_info_snap' not in locals(): # S'assurer que la version snapshot est définie
    def get_pivot_info_snap(pivot_series):
         if pivot_series is None or not isinstance(pivot_series, pd.Series): return {'time': pd.NaT, 'price': np.nan, 'time_numeric': np.nan, 'original_index': -1}
         time_numeric_val = pivot_series.get('time_numeric', np.nan)
         if pd.isna(time_numeric_val) and not pd.isna(pivot_series.get('time')):
             try:
                 dt_obj = pivot_series['time']
                 if isinstance(dt_obj, pd.Timestamp): dt_obj = dt_obj.to_pydatetime()
                 time_numeric_val = dt_obj.timestamp()
             except Exception as e: time_numeric_val = np.nan
         return {'time': pivot_series.get('time', pd.NaT), 'price': pivot_series.get('price', np.nan), 'time_numeric': time_numeric_val, 'original_index': pivot_series.name if hasattr(pivot_series, 'name') else -1}

def calculate_all_channels_at_date(end_date_dt, full_history_df, config, zigzag_threshold=0.05):
    """
    Calcule les pivots ZigZag et les canaux Macro, Meso, Micro avec logique de fallback
    pour les dates de début de Meso et Micro.
    """
    try:
        # 1. Filtrer l'historique
        hist_for_calc = full_history_df[full_history_df['time'] <= end_date_dt].copy()
        if len(hist_for_calc) < 5: return None, None

        # 2. Calculer les pivots ZigZag
        snap_pivots_list = classic_chart_zigzag(hist_for_calc[['time','close']], thresholdPercent=zigzag_threshold)
        if not snap_pivots_list or len(snap_pivots_list) < 2: return snap_pivots_list, None

        snap_pivots_df = pd.DataFrame(snap_pivots_list, columns=['time', 'price', 'type'])
        snap_pivots_df['time_numeric'] = snap_pivots_df['time'].apply(lambda x: x.timestamp())
        snap_high_pivots = snap_pivots_df[snap_pivots_df['type'] == -1].copy()
        snap_low_pivots = snap_pivots_df[snap_pivots_df['type'] == +1].copy()
        snap_all_high_np = snap_high_pivots[['time_numeric', 'price']].values
        snap_all_low_np = snap_low_pivots[['time_numeric', 'price']].values

        if len(snap_high_pivots) < 2 or len(snap_low_pivots) < 2: return snap_pivots_list, None

        # 3. Calculer les canaux hiérarchiques
        channels = {"macro": {}, "meso": {}, "micro": {}}
        # Initialiser avec None pour faciliter la détection d'échec
        for scale in channels:
            channels[scale]["resistance"] = (None, None)
            channels[scale]["support"] = (None, None)

        # --- Macro ---
        macro_success = False
        try:
            res1_m, res2_m = find_best_channel_line_strict_weighted(snap_high_pivots, snap_all_high_np, True, config['wp_macro_res'], config['rpf_macro_res'])
            sup1_m, sup2_m = find_best_channel_line_strict_weighted(snap_low_pivots, snap_all_low_np, False, config['wp_macro_sup'], config['rpf_macro_sup'])
            # Stocker les infos seulement si les deux pivots sont trouvés
            if res1_m is not None and res2_m is not None:
                 channels["macro"]["resistance"] = (get_pivot_info_snap(res1_m), get_pivot_info_snap(res2_m))
            if sup1_m is not None and sup2_m is not None:
                 channels["macro"]["support"] = (get_pivot_info_snap(sup1_m), get_pivot_info_snap(sup2_m))
            # Macro est un succès si AU MOINS un support ET une résistance sont définis
            if channels["macro"]["resistance"][0] is not None and channels["macro"]["support"][0] is not None:
                macro_success = True
        except Exception as e_macro: pass

        # --- Meso (avec Fallback) ---
        meso_success = False
        if macro_success:
            macro_pivots_info = [
                channels["macro"]["resistance"][0], channels["macro"]["resistance"][1],
                channels["macro"]["support"][0], channels["macro"]["support"][1]
            ]
            # Filtrer les pivots valides (non None et avec une date)
            valid_macro_pivots = [p for p in macro_pivots_info if p and not pd.isna(p.get('time'))]
            valid_macro_pivots.sort(key=lambda p: pd.to_datetime(p['time']), reverse=True) # Trier par date décroissante

            meso_start_time = None
            # Essai 1: 2ème pivot le plus récent
            if len(valid_macro_pivots) >= 2:
                meso_start_time = pd.to_datetime(valid_macro_pivots[1]['time']) # 2ème plus récent

            # Essai 2 (Fallback): 3ème pivot le plus récent (si essai 1 échoue et si assez de pivots)
            attempted_fallback_meso = False
            if meso_start_time is not None:
                meso_high_f = snap_high_pivots[snap_high_pivots['time'] >= meso_start_time].copy()
                meso_low_f = snap_low_pivots[snap_low_pivots['time'] >= meso_start_time].copy()
                # Si pas assez de pivots après le 2ème, et si on a au moins 4 pivots macro pour tenter le 3ème
                if (len(meso_high_f) < 2 or len(meso_low_f) < 2) and len(valid_macro_pivots) >= 4:
                    meso_start_time = pd.to_datetime(valid_macro_pivots[2]['time']) # 3ème plus récent
                    attempted_fallback_meso = True
            elif len(valid_macro_pivots) >= 3: # Si l'essai 1 n'a même pas pu avoir lieu (moins de 2 pivots) mais qu'on en a 3 ou 4
                 meso_start_time = pd.to_datetime(valid_macro_pivots[2]['time']) # Essayer directement le 3ème
                 attempted_fallback_meso = True


            # Calcul Meso si une date de début valide a été trouvée
            if meso_start_time is not None:
                 try:
                    meso_high_f = snap_high_pivots[snap_high_pivots['time'] >= meso_start_time].copy()
                    meso_low_f = snap_low_pivots[snap_low_pivots['time'] >= meso_start_time].copy()
                    if len(meso_high_f) >= 2 and len(meso_low_f) >= 2:
                        meso_high_np = meso_high_f[['time_numeric', 'price']].values
                        meso_low_np = meso_low_f[['time_numeric', 'price']].values
                        res1_me, res2_me = find_best_channel_line_strict_weighted(meso_high_f, meso_high_np, True, config['wp_meso_res'], config['rpf_meso_res'])
                        sup1_me, sup2_me = find_best_channel_line_strict_weighted(meso_low_f, meso_low_np, False, config['wp_meso_sup'], config['rpf_meso_sup'])
                        if res1_me is not None and res2_me is not None:
                            channels["meso"]["resistance"] = (get_pivot_info_snap(res1_me), get_pivot_info_snap(res2_me))
                        if sup1_me is not None and sup2_me is not None:
                            channels["meso"]["support"] = (get_pivot_info_snap(sup1_me), get_pivot_info_snap(sup2_me))
                        # Meso succès si résistance ET support trouvés
                        if channels["meso"]["resistance"][0] is not None and channels["meso"]["support"][0] is not None:
                             meso_success = True
                             # print(f"DEBUG [{end_date_dt.date()}] Meso OK {'(Fallback)' if attempted_fallback_meso else ''}")
                    # else:
                         # print(f"DEBUG [{end_date_dt.date()}] Meso FAIL: Not enough pivots after start {'(Fallback)' if attempted_fallback_meso else ''}")

                 except Exception as e_meso: pass # print(f"DEBUG [{end_date_dt.date()}] Meso Calc Error: {e_meso}")
            # else:
                 # print(f"DEBUG [{end_date_dt.date()}] Meso FAIL: No valid start time found.")


        # --- Micro (avec Fallback) ---
        # Note: On ne définit pas micro_success, on calcule juste si possible
        if meso_success: # Micro ne peut exister que si Meso a réussi
            meso_pivots_info = [
                channels["meso"]["resistance"][0], channels["meso"]["resistance"][1],
                channels["meso"]["support"][0], channels["meso"]["support"][1]
            ]
            valid_meso_pivots = [p for p in meso_pivots_info if p and not pd.isna(p.get('time'))]
            valid_meso_pivots.sort(key=lambda p: pd.to_datetime(p['time']), reverse=True)

            micro_start_time = None
            # Essai 1: 2ème pivot meso le plus récent
            if len(valid_meso_pivots) >= 2:
                micro_start_time = pd.to_datetime(valid_meso_pivots[1]['time'])

            # Essai 2 (Fallback): 3ème pivot meso le plus récent
            attempted_fallback_micro = False
            if micro_start_time is not None:
                 micro_high_f_test = snap_high_pivots[snap_high_pivots['time'] >= micro_start_time].copy()
                 micro_low_f_test = snap_low_pivots[snap_low_pivots['time'] >= micro_start_time].copy()
                 if (len(micro_high_f_test) < 2 or len(micro_low_f_test) < 2) and len(valid_meso_pivots) >= 4:
                      micro_start_time = pd.to_datetime(valid_meso_pivots[2]['time'])
                      attempted_fallback_micro = True
            elif len(valid_meso_pivots) >= 3:
                 micro_start_time = pd.to_datetime(valid_meso_pivots[2]['time'])
                 attempted_fallback_micro = True


            # Calcul Micro si une date de début valide a été trouvée
            if micro_start_time is not None:
                try:
                    micro_high_f = snap_high_pivots[snap_high_pivots['time'] >= micro_start_time].copy()
                    micro_low_f = snap_low_pivots[snap_low_pivots['time'] >= micro_start_time].copy()
                    if len(micro_high_f) >= 2 and len(micro_low_f) >= 2:
                        micro_high_np = micro_high_f[['time_numeric', 'price']].values
                        micro_low_np = micro_low_f[['time_numeric', 'price']].values
                        res1_mi, res2_mi = find_best_channel_line_strict_weighted(micro_high_f, micro_high_np, True, config['wp_micro_res'], config['rpf_micro_res'])
                        sup1_mi, sup2_mi = find_best_channel_line_strict_weighted(micro_low_f, micro_low_np, False, config['wp_micro_sup'], config['rpf_micro_sup'])
                        if res1_mi is not None and res2_mi is not None:
                             channels["micro"]["resistance"] = (get_pivot_info_snap(res1_mi), get_pivot_info_snap(res2_mi))
                        if sup1_mi is not None and sup2_mi is not None:
                            channels["micro"]["support"] = (get_pivot_info_snap(sup1_mi), get_pivot_info_snap(sup2_mi))
                        # if channels["micro"]["resistance"][0] or channels["micro"]["support"][0]:
                        #     print(f"DEBUG [{end_date_dt.date()}] Micro OK {'(Fallback)' if attempted_fallback_micro else ''}")
                    # else:
                        # print(f"DEBUG [{end_date_dt.date()}] Micro FAIL: Not enough pivots after start {'(Fallback)' if attempted_fallback_micro else ''}")

                except Exception as e_micro: pass # print(f"DEBUG [{end_date_dt.date()}] Micro Calc Error: {e_micro}")
            # else:
                # print(f"DEBUG [{end_date_dt.date()}] Micro FAIL: No valid start time found.")

        return snap_pivots_list, channels

    except Exception as e:
        print(f"ERREUR Majeure dans calculate_all_channels_at_date pour {end_date_dt}: {e}")
        # traceback.print_exc()
        return None, None


In [44]:
# CELLULE 29 : Calcul des Snapshots Historiques (Révisée)

import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import traceback # Importé pour le debug au besoin

# --- Vérifier les dépendances clés ---
if 'bars_df' not in locals(): raise NameError("bars_df non défini")
if 'final_config' not in locals(): raise NameError("final_config non définie")
if 'calculate_all_channels_at_date' not in locals() or not callable(calculate_all_channels_at_date):
    raise NameError("Fonction 'calculate_all_channels_at_date' non définie ou non appelable.")

# --- Dates de Snapshot (Logique de génération inchangée) ---
data_tz = bars_df['time'].dt.tz # Récupérer le timezone des données sources

# Utiliser une date de début pour les snapshots (ex: 1 an avant la fin des données ou date spécifique)
snapshot_start_calc_date = bars_df['time'].iloc[0] # Ou une date spécifique: pd.Timestamp("2024-01-01", tz=data_tz)
if snapshot_start_calc_date > bars_df['time'].iloc[-1] - pd.Timedelta(days=60): # S'assurer qu'on a au moins 2 mois de snapshots
     snapshot_start_calc_date = bars_df['time'].iloc[-1] - pd.Timedelta(days=60)

last_hist_date = bars_df['time'].iloc[-1]

# Générer les dates de fin de mois (ou autre fréquence)
freq_snapshots = 'M' # 'M'=Fin de Mois, 'W'=Fin de Semaine, 'D'=Jour etc.
print(f"Génération des dates de snapshots (freq='{freq_snapshots}') entre {snapshot_start_calc_date.date()} et {last_hist_date.date()}")
try:
    # Générer les dates de fin de période
    snapshot_dates_gen = pd.date_range(
        start=snapshot_start_calc_date,
        end=last_hist_date,
        freq=freq_snapshots,
        tz=data_tz # Appliquer le timezone des données sources
    )
    # S'assurer que les dates générées ne dépassent pas la dernière date historique
    snapshot_dates_gen = snapshot_dates_gen[snapshot_dates_gen <= last_hist_date]

except Exception as e:
    print(f"ERREUR lors de la génération de date_range: {e}")
    snapshot_dates = pd.DatetimeIndex([]) # Créer un index vide pour éviter erreur aval
    # raise # Optionnel: arrêter l'exécution ici

if not snapshot_dates_gen.empty:
     # Ajouter la toute dernière date de l'historique si elle est significativement après la dernière date générée
    if last_hist_date > snapshot_dates_gen[-1] + pd.Timedelta(hours=1):
         snapshot_dates_list = snapshot_dates_gen.to_list()
         snapshot_dates_list.append(last_hist_date)
         snapshot_dates = pd.DatetimeIndex(snapshot_dates_list).sort_values()
    else:
         snapshot_dates = snapshot_dates_gen
    # S'assurer qu'il n'y a pas de doublons (si last_hist_date tombe sur une fin de période)
    snapshot_dates = snapshot_dates.unique()
else: # Si date_range est vide ou période très courte
     snapshot_dates = pd.DatetimeIndex([last_hist_date], tz=data_tz) # Garder le timezone


print(f"Calcul pour {len(snapshot_dates)} dates de snapshots : {snapshot_dates.strftime('%Y-%m-%d').tolist()}")

# --- Calcul Effectif des Snapshots ---
snapshot_results = {}
config_for_snapshots = final_config # Utiliser la configuration finale sélectionnée

print(f"\nUtilisation de final_config '{config_for_snapshots.get('label', 'N/A')}' pour les calculs de snapshots.")

for snap_date in tqdm(snapshot_dates, desc="Calculating Snapshots"):
    # La fonction `calculate_all_channels_at_date` gère le filtrage de l'historique
    # S'assurer que la date passée a le bon TZ ou est naive si les données sont naives
    snap_date_to_pass = snap_date
    if data_tz is None and getattr(snap_date, 'tz', None) is not None:
        snap_date_to_pass = snap_date.tz_convert(None)
    elif data_tz is not None and getattr(snap_date, 'tz', None) is None:
         try:
             snap_date_to_pass = snap_date.tz_localize(data_tz)
         except Exception as e:
              print(f"Warn: Could not localize snap_date {snap_date} to {data_tz}. Skipping snapshot.")
              snapshot_results[snap_date] = {"pivots": None, "channels": None}
              continue # Passer au snapshot suivant

    # Appeler la fonction de calcul dédiée
    pivots, channels = calculate_all_channels_at_date(snap_date_to_pass, bars_df, config_for_snapshots)

    # Stocker les résultats (même si channels est None, on garde les pivots s'ils existent)
    snapshot_results[snap_date] = {"pivots": pivots, "channels": channels}

print("\nCalcul des snapshots terminé.")

# Optionnel: Vérifier combien de snapshots ont réussi à calculer des canaux
successful_channel_calcs = sum(1 for r in snapshot_results.values() if r and r.get('channels') is not None)
print(f"{successful_channel_calcs} / {len(snapshot_dates)} snapshots ont pu calculer au moins une partie des canaux.")


### 6.2. Analyse de Stabilité des Pivots (Optionnel)

Cette cellule analyse les résultats des snapshots (`snapshot_results`) pour quantifier la fréquence à laquelle les pivots définissant chaque ligne de canal (Macro, Méso, Micro - Support & Résistance) changent d'un snapshot à l'autre. Une faible fréquence de changement suggère une plus grande robustesse des canaux identifiés.

In [45]:
# CELLULE 29b (Nouvelle) : Analyse de la Stabilité des Pivots Définissant les Canaux

import pandas as pd

if 'snapshot_results' not in locals() or not snapshot_results:
    print("ERREUR: Pas de résultats de snapshots à analyser.")
else:
    stability_data = []
    previous_pivots = {} # Stocker les pivots du snapshot précédent

    for snap_date in sorted(snapshot_results.keys()):
        result = snapshot_results[snap_date]
        if not result or not result.get('channels'):
            continue

        channels = result['channels']
        current_pivots = {}
        row_data = {'snapshot_date': snap_date}

        for scale in ['macro', 'meso', 'micro']:
            for line_type in ['support', 'resistance']:
                key = f"{scale}_{line_type}"
                p1_info, p2_info = channels.get(scale, {}).get(line_type, ({}, {}))

                # Utiliser le timestamp comme identifiant unique (plus robuste que l'index)
                p1_id = pd.to_datetime(p1_info.get('time')).isoformat() if isinstance(p1_info, dict) and not pd.isna(p1_info.get('time')) else None
                p2_id = pd.to_datetime(p2_info.get('time')).isoformat() if isinstance(p2_info, dict) and not pd.isna(p2_info.get('time')) else None
                current_pivots[key] = (p1_id, p2_id)

                row_data[f"{key}_p1_time"] = p1_id
                row_data[f"{key}_p2_time"] = p2_id

                # Comparer avec le snapshot précédent
                changed = False
                if key in previous_pivots:
                    if previous_pivots[key] != current_pivots[key]:
                        changed = True
                row_data[f"{key}_changed"] = changed

        stability_data.append(row_data)
        previous_pivots = current_pivots # Mettre à jour pour la prochaine itération

    if stability_data:
        stability_df = pd.DataFrame(stability_data)
        stability_df = stability_df.set_index('snapshot_date')

        print("\n--- Analyse de Stabilité des Pivots Définissant les Canaux ---")
        # Afficher les colonnes indiquant les changements
        change_cols = [col for col in stability_df.columns if 'changed' in col]
        if change_cols:
             print(stability_df[change_cols].to_string())
             # Compter le nombre total de changements pour chaque canal
             print("\nNombre total de changements de pivots par canal:")
             print(stability_df[change_cols].sum())
        else:
             print("Aucune colonne de changement trouvée.")

    else:
        print("Impossible de générer les données de stabilité.")

### 6.3. Visualisation des Snapshots (Configuration Cible)

Ce graphique présente les résultats de l'analyse temporelle (basée sur `final_config`) sous forme de grille de sous-graphiques (subplots).

**Organisation de la Grille :**

*   Chaque sous-graphique correspond à une **date de snapshot**.
*   Le titre indique la date de fin de l'historique utilisé.
*   Chaque sous-graphique affiche :
    *   Prix de clôture jusqu'au snapshot.
    *   Pivots ZigZag du snapshot.
    *   Canaux Macro, Méso et Micro déterminés à cette date.
*   Chaque sous-graphique **zoome automatiquement** sur une fenêtre temporelle et une plage de prix définies par les **pivots des canaux Meso et Micro** (si disponibles, sinon Macro) calculés pour ce snapshot spécifique, pour mieux visualiser la structure récente.

Permet d'observer comment la structure hiérarchique (calculée avec `final_config`) a **évolué au fil du temps**.

In [46]:
# CELLULE 30 (MODIFIÉE) : Visualisation 2D Snapshots - Zoom Meso/Micro

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import math

# --- Pré-requis ---
if 'snapshot_results' not in locals() or not snapshot_results: print("ERREUR: Pas de résultats de snapshots ('snapshot_results') à visualiser.")
elif 'final_config' not in locals(): print("ERREUR: `final_config` non définie.")
elif 'get_line_params_time' not in locals(): print("ERREUR: `get_line_params_time` non définie.")
elif 'bars_df' not in locals(): print("ERREUR: `bars_df` non disponible.")
elif not any(res is not None and res.get('channels') is not None for date, res in snapshot_results.items() if res is not None):
    print("WARN: Aucun snapshot n'a pu calculer de canaux valides. Impossible de générer le graphique.")
else:
    valid_snapshots = {date: res for date, res in snapshot_results.items() if res and res.get('channels') is not None}
    if not valid_snapshots: print("ERREUR: Aucun snapshot avec des canaux valides trouvé pour le plot.")
    else:
        sorted_snap_dates = sorted(valid_snapshots.keys())
        n_snapshots_to_plot = len(sorted_snap_dates)
        print(f"Nombre de snapshots valides à afficher: {n_snapshots_to_plot}")

        if n_snapshots_to_plot > 0:
            # --- Configuration Grille ---
            cols = 4; rows = math.ceil(n_snapshots_to_plot / cols)
            print(f"Affichage de {n_snapshots_to_plot} snapshots sur une grille de {rows}x{cols}.")
            subplot_titles=[d.strftime('%Y-%m-%d') for d in sorted_snap_dates]
            subplot_titles.extend([''] * (rows * cols - len(subplot_titles)))
            fig = make_subplots(rows=rows, cols=cols, subplot_titles=subplot_titles,
                                shared_xaxes=False, shared_yaxes=False,
                                vertical_spacing=0.08, horizontal_spacing=0.05)
            channel_colors = {"macro": "rgba(60, 60, 60, 0.8)", "meso": "rgba(0, 0, 255, 0.6)", "micro": "rgba(255, 0, 255, 0.7)"}
            channel_styles = {"macro": "solid", "meso": "dash", "micro": "dot"}
            data_tz = bars_df['time'].dt.tz

            # --- Boucle de Plotting ---
            for i, snap_date in enumerate(sorted_snap_dates):
                row_idx = (i // cols) + 1; col_idx = (i % cols) + 1
                result = valid_snapshots[snap_date]
                snap_pivots_list = result.get("pivots")
                snap_channels = result.get("channels")

                # --- Calcul Zoom X/Y (NOUVELLE LOGIQUE: Priorité Meso/Micro) ---
                zoom_pivots_data = []
                yaxis_range_subplot = None
                xaxis_range_subplot = None
                zoom_level_used = "Fallback" # Pour info

                if snap_channels:
                    # 1. Essayer de collecter les pivots Meso ET Micro
                    for scale in ["meso", "micro"]:
                        scale_def = snap_channels.get(scale, {})
                        res_p12 = scale_def.get("resistance", ({},{}))
                        sup_p12 = scale_def.get("support", ({},{}))
                        for p_info in [res_p12[0], res_p12[1], sup_p12[0], sup_p12[1]]:
                            if isinstance(p_info, dict) and not pd.isna(p_info.get('time')) and not pd.isna(p_info.get('price')):
                                 time_val = pd.to_datetime(p_info['time']).to_pydatetime()
                                 if getattr(time_val, 'tzinfo', None) is not None: time_val = time_val.replace(tzinfo=None)
                                 zoom_pivots_data.append({'time': time_val, 'price': p_info['price']})

                    if len(zoom_pivots_data) >= 2:
                         zoom_level_used = "Meso/Micro"
                    else:
                         # 2. Si échec, essayer avec Macro seulement
                         zoom_pivots_data = [] # Reset
                         scale_def = snap_channels.get("macro", {})
                         res_p12 = scale_def.get("resistance", ({},{}))
                         sup_p12 = scale_def.get("support", ({},{}))
                         for p_info in [res_p12[0], res_p12[1], sup_p12[0], sup_p12[1]]:
                              if isinstance(p_info, dict) and not pd.isna(p_info.get('time')) and not pd.isna(p_info.get('price')):
                                   time_val = pd.to_datetime(p_info['time']).to_pydatetime()
                                   if getattr(time_val, 'tzinfo', None) is not None: time_val = time_val.replace(tzinfo=None)
                                   zoom_pivots_data.append({'time': time_val, 'price': p_info['price']})
                         if len(zoom_pivots_data) >= 2:
                              zoom_level_used = "Macro"

                # 3. Calculer les ranges à partir des pivots collectés (Meso/Micro ou Macro)
                if len(zoom_pivots_data) >= 2:
                    zoom_pivots_df = pd.DataFrame(zoom_pivots_data)
                    min_p_zoom = zoom_pivots_df['price'].min(); max_p_zoom = zoom_pivots_df['price'].max()
                    min_t_zoom = zoom_pivots_df['time'].min(); max_t_zoom = zoom_pivots_df['time'].max()

                    height_zoom = max_p_zoom - min_p_zoom
                    margin_y = max(height_zoom * 0.30, (min_p_zoom + max_p_zoom)/2 * 0.05) # Marge Y 30%
                    yaxis_range_subplot = [min_p_zoom - margin_y, max_p_zoom + margin_y]

                    min_t_zoom_ts = pd.Timestamp(min_t_zoom, tz=data_tz); max_t_zoom_ts = pd.Timestamp(max_t_zoom, tz=data_tz)
                    duration_zoom = max_t_zoom_ts - min_t_zoom_ts if max_t_zoom_ts > min_t_zoom_ts else pd.Timedelta(days=1)
                    # Ajuster marges temporelles pour bien voir la structure ciblée
                    margin_t_before = max(duration_zoom * 0.5, pd.Timedelta(days=45)) # Marge avant 50%
                    margin_t_after = max(duration_zoom * 0.2, pd.Timedelta(days=20))  # Marge après 20%
                    xaxis_range_subplot = [min_t_zoom_ts - margin_t_before, max_t_zoom_ts + margin_t_after]
                    xaxis_range_subplot[1] = min(xaxis_range_subplot[1], pd.Timestamp(snap_date + pd.Timedelta(days=5), tz=data_tz))
                    # print(f"DEBUG Zoom {snap_date.date()}: Used {zoom_level_used} pivots.") # Décommenter pour vérifier

                # 4. Si AUCUN pivot trouvé pour zoomer (Meso/Micro ET Macro échouent) -> Fallback temporel
                if xaxis_range_subplot is None:
                    zoom_level_used = "Fallback Time"
                    lookback_plot_days = 90
                    xaxis_range_subplot = [pd.Timestamp(snap_date - pd.Timedelta(days=lookback_plot_days), tz=data_tz), pd.Timestamp(snap_date + pd.Timedelta(days=5), tz=data_tz)]
                    bars_subplot_fallback = bars_df[(bars_df['time'] >= xaxis_range_subplot[0]) & (bars_df['time'] <= snap_date)].copy()
                    if not bars_subplot_fallback.empty:
                         min_p = bars_subplot_fallback['low'].min(); max_p = bars_subplot_fallback['high'].max(); margin_y = (max_p - min_p) * 0.1 if (max_p - min_p) > 1e-6 else max_p * 0.05
                         yaxis_range_subplot = [min_p - margin_y, max_p + margin_y]
                    # print(f"DEBUG Zoom {snap_date.date()}: Used Fallback Time.") # Décommenter pour vérifier
                # --- Fin Calcul Zoom X/Y ---

                # --- Filtrage Données et Tracé (reste identique à la version précédente de Cell 30) ---
                # ... (coller ici la partie filtrage et tracé de la cellule 30 précédente) ...
                # ... (Assurez-vous d'utiliser xaxis_range_subplot et yaxis_range_subplot calculés ci-dessus) ...

                # 1. Filtrage
                bars_subplot = bars_df[(bars_df['time'] >= xaxis_range_subplot[0]) & (bars_df['time'] <= snap_date)].copy()
                pivots_in_view_df = pd.DataFrame()
                if snap_pivots_list:
                    pivots_plot_df = pd.DataFrame(snap_pivots_list, columns=['time','price','type']); pivots_plot_df['time'] = pd.to_datetime(pivots_plot_df['time'])
                    pivot_tz = pivots_plot_df['time'].dt.tz
                    if data_tz is not None and pivot_tz is None:
                         try: pivots_plot_df['time'] = pivots_plot_df['time'].dt.tz_localize(data_tz, ambiguous='infer', nonexistent='shift_forward')
                         except: pass
                    elif data_tz is None and pivot_tz is not None: pivots_plot_df['time'] = pivots_plot_df['time'].dt.tz_convert(None)
                    elif data_tz is not None and pivot_tz is not None and hasattr(data_tz, 'zone') and hasattr(pivot_tz, 'zone') and data_tz.zone != pivot_tz.zone: pivots_plot_df['time'] = pivots_plot_df['time'].dt.tz_convert(data_tz)
                    pivots_in_view_df = pivots_plot_df[(pivots_plot_df['time'] >= xaxis_range_subplot[0]) & (pivots_plot_df['time'] <= snap_date)]
                if yaxis_range_subplot is None: # Fallback Y si besoin
                     all_prices_in_view = pd.concat([bars_subplot['low'], bars_subplot['high'], pivots_in_view_df['price']]).dropna()
                     if not all_prices_in_view.empty:
                         min_p = all_prices_in_view.min(); max_p = all_prices_in_view.max(); margin_y = (max_p - min_p) * 0.1 if (max_p - min_p) > 1e-6 else max_p * 0.05
                         yaxis_range_subplot = [min_p - margin_y, max_p + margin_y]

                # 2. Tracé Prix
                if not bars_subplot.empty: fig.add_trace(go.Scatter(x=bars_subplot['time'], y=bars_subplot['close'], mode='lines', name='Close', line=dict(color='rgba(150, 150, 150, 0.5)'), showlegend=(i==0)), row=row_idx, col=col_idx)

                # 3. Tracé Pivots
                if not pivots_in_view_df.empty:
                    high_x = pivots_in_view_df[pivots_in_view_df['type'] < 0]['time']; high_y = pivots_in_view_df[pivots_in_view_df['type'] < 0]['price']
                    low_x  = pivots_in_view_df[pivots_in_view_df['type'] > 0]['time']; low_y  = pivots_in_view_df[pivots_in_view_df['type'] > 0]['price']
                    fig.add_trace(go.Scatter(x=high_x, y=high_y, mode='markers', name='High Pivot', marker=dict(color='red', size=5, symbol='diamond-open'), showlegend=(i==0)), row=row_idx, col=col_idx)
                    fig.add_trace(go.Scatter(x=low_x, y=low_y, mode='markers', name='Low Pivot', marker=dict(color='green', size=5, symbol='circle-open'), showlegend=(i==0)), row=row_idx, col=col_idx)

                # 4. Tracé Canaux
                if snap_channels and not bars_subplot.empty:
                    plot_start_time_eff = bars_subplot['time'].min(); plot_end_time_eff = snap_date
                    plot_start_time_num_eff = plot_start_time_eff.timestamp(); plot_end_time_num_eff = plot_end_time_eff.timestamp()
                    for channel_name in ["macro", "meso", "micro"]:
                         definition = snap_channels.get(channel_name, {})
                         for line_type in ["resistance", "support"]:
                             p1_info, p2_info = definition.get(line_type, ({}, {}))
                             if (isinstance(p1_info, dict) and isinstance(p2_info, dict) and # Check validité
                                 not pd.isna(p1_info.get('time')) and not pd.isna(p2_info.get('time')) and
                                 not pd.isna(p1_info.get('time_numeric')) and not pd.isna(p2_info.get('time_numeric')) and
                                 not pd.isna(p1_info.get('price')) and not pd.isna(p2_info.get('price'))):
                                 p1_time_num = p1_info['time_numeric']; p2_time_num = p2_info['time_numeric']
                                 p1_price = p1_info['price']; p2_price = p2_info['price']
                                 p1_time = pd.to_datetime(p1_info['time']); p2_time = pd.to_datetime(p2_info['time'])
                                 try:
                                     m, c = get_line_params_time(p1_time_num, p1_price, p2_time_num, p2_price)
                                     if m != np.inf:
                                         line_plot_start_time_dt = max(p1_time, plot_start_time_eff); line_plot_start_time_num = line_plot_start_time_dt.timestamp()
                                         line_plot_end_time_dt = plot_end_time_eff
                                         line_start_y_plot = m * line_plot_start_time_num + c; line_end_y_plot = m * plot_end_time_num_eff + c
                                         fig.add_trace(go.Scatter(x=[line_plot_start_time_dt, line_plot_end_time_dt], y=[line_start_y_plot, line_end_y_plot], mode='lines', name=f"{channel_name.capitalize()} {line_type.capitalize()}", line=dict(color=channel_colors[channel_name], width=1.5, dash=channel_styles[channel_name]), legendgroup=f"{channel_name}_{line_type}", showlegend=(i==0)), row=row_idx, col=col_idx)
                                         pivots_x_to_mark = []; pivots_y_to_mark = []
                                         if xaxis_range_subplot and p1_time >= xaxis_range_subplot[0] and p1_time <= xaxis_range_subplot[1]: pivots_x_to_mark.append(p1_time); pivots_y_to_mark.append(p1_price)
                                         if xaxis_range_subplot and p2_time >= xaxis_range_subplot[0] and p2_time <= xaxis_range_subplot[1]: pivots_x_to_mark.append(p2_time); pivots_y_to_mark.append(p2_price)
                                         if pivots_x_to_mark: fig.add_trace(go.Scatter(x=pivots_x_to_mark, y=pivots_y_to_mark, mode='markers', marker=dict(color=channel_colors[channel_name], size=7, symbol='star' if line_type == 'resistance' else 'star-open'), showlegend=False, hoverinfo='skip'), row=row_idx, col=col_idx)
                                 except Exception as e_plotline: pass # print(f"WARN: Plot line {snap_date.date()}: {e_plotline}")


                # --- Application Zoom et Mise en Forme Axes ---
                if yaxis_range_subplot: fig.update_yaxes(range=yaxis_range_subplot, row=row_idx, col=col_idx)
                if xaxis_range_subplot: fig.update_xaxes(range=xaxis_range_subplot, row=row_idx, col=col_idx)
                if row_idx < rows: fig.update_xaxes(showticklabels=False, row=row_idx, col=col_idx)
                if col_idx > 1: fig.update_yaxes(showticklabels=False, row=row_idx, col=col_idx)

            # --- Layout Final ---
            fig.update_layout(
                title=f"Évolution Temporelle des Canaux ({n_snapshots_to_plot} Snapshots - Config: {final_config.get('label','N/A')}) - Zoom Meso/Micro",
                height=250 * rows + 80, width=1200, hovermode='x unified',
                legend=dict(orientation="h", yanchor="bottom", y=-0.08, xanchor="center", x=0.5)
            )
            for i_annot, annot in enumerate(fig.layout.annotations):
                 if i_annot < n_snapshots_to_plot: annot.update(font=dict(size=9))
            fig.show()

### 6.4. Visualisation des Snapshots (Configurations Multiples)

Similaire à la visualisation précédente, mais cette cellule génère une grille de snapshots **pour plusieurs configurations de canaux sélectionnées** (définies par `indices_configs_snapshots`).

Cela permet de comparer directement comment différentes configurations de `wp`/`rpf` auraient influencé l'évolution historique perçue des canaux. Chaque configuration génère son propre graphique de snapshots complet. Le zoom est également basé sur les pivots Meso/Micro (ou Macro en fallback) pour chaque snapshot de chaque configuration.

In [ ]:
# CELLULE 30b (MODIFIÉE) : Affichage Snapshots pour Plusieurs Configs - Zoom Meso/Micro

import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import traceback

# --- Indices des Configurations à Visualiser ---
indices_configs_snapshots = [3, 7, 11] # <--- !!! UTILISATEUR: MODIFIER CETTE LISTE D'INDICES !!!

# --- Vérifications ---
if 'configurations_to_explore' not in locals(): raise NameError("configurations_to_explore non définie")
if 'bars_df' not in locals(): raise NameError("bars_df non défini")
if 'calculate_all_channels_at_date' not in locals() or not callable(calculate_all_channels_at_date): raise NameError("calculate_all_channels_at_date non définie")
if 'get_line_params_time' not in locals(): raise NameError("get_line_params_time non définie")

# --- Boucle sur les Configurations ---
for config_idx in indices_configs_snapshots:
    if not (0 <= config_idx < len(configurations_to_explore)):
        print(f"WARN: Index {config_idx} invalide. Skip."); continue

    config_to_snapshot = configurations_to_explore[config_idx]
    config_label = config_to_snapshot.get('label', f'Config {config_idx}')
    print(f"\n{'='*10} Génération Snapshots pour Config: {config_label} (Index {config_idx}) {'='*10}")

    # --- 1. RECALCUL SNAPSHOTS pour CETTE config ---
    current_config_snapshot_results = {}
    data_tz = bars_df['time'].dt.tz
    snapshot_start_calc_date_cfg = bars_df['time'].iloc[0]
    if snapshot_start_calc_date_cfg > bars_df['time'].iloc[-1] - pd.Timedelta(days=60): snapshot_start_calc_date_cfg = bars_df['time'].iloc[-1] - pd.Timedelta(days=60)
    last_hist_date_cfg = bars_df['time'].iloc[-1]
    freq_snapshots_cfg = 'M'
    try:
        snapshot_dates_gen_cfg = pd.date_range(start=snapshot_start_calc_date_cfg, end=last_hist_date_cfg, freq=freq_snapshots_cfg, tz=data_tz)
        snapshot_dates_gen_cfg = snapshot_dates_gen_cfg[snapshot_dates_gen_cfg <= last_hist_date_cfg]
    except Exception as e_dr: print(f"ERREUR date_range {config_label}: {e_dr}"); continue
    if not snapshot_dates_gen_cfg.empty:
         if last_hist_date_cfg > snapshot_dates_gen_cfg[-1] + pd.Timedelta(hours=1):
             snapshot_dates_list_cfg = snapshot_dates_gen_cfg.to_list(); snapshot_dates_list_cfg.append(last_hist_date_cfg)
             snapshot_dates_cfg = pd.DatetimeIndex(snapshot_dates_list_cfg).sort_values().unique()
         else: snapshot_dates_cfg = snapshot_dates_gen_cfg.unique()
    else: snapshot_dates_cfg = pd.DatetimeIndex([last_hist_date_cfg], tz=data_tz)
    print(f"Calcul pour {len(snapshot_dates_cfg)} snapshots pour {config_label}...")
    for snap_date_cfg in tqdm(snapshot_dates_cfg, desc=f"Calc Snapshots {config_label[:15]}...", leave=False):
         snap_date_to_pass_cfg = snap_date_cfg
         if data_tz is None and getattr(snap_date_cfg, 'tz', None) is not None: snap_date_to_pass_cfg = snap_date_cfg.tz_convert(None)
         elif data_tz is not None and getattr(snap_date_cfg, 'tz', None) is None:
              try: snap_date_to_pass_cfg = snap_date_cfg.tz_localize(data_tz)
              except: print(f"Warn TZ loc snap {config_label}"); continue
         pivots, channels = calculate_all_channels_at_date(snap_date_to_pass_cfg, bars_df, config_to_snapshot) # Utilise config_to_snapshot
         current_config_snapshot_results[snap_date_cfg] = {"pivots": pivots, "channels": channels}
    print(f"Calcul snapshots pour {config_label} terminé.")

    # --- 2. AFFICHAGE GRAPHIQUE pour CETTE config ---
    valid_snapshots_cfg = {date: res for date, res in current_config_snapshot_results.items() if res and res.get('channels') is not None}
    if not valid_snapshots_cfg:
        print(f"Aucun snapshot valide pour {config_label}. Skip plot."); continue

    sorted_snap_dates_cfg = sorted(valid_snapshots_cfg.keys())
    n_snapshots_to_plot_cfg = len(sorted_snap_dates_cfg)
    cols_cfg = 4; rows_cfg = math.ceil(n_snapshots_to_plot_cfg / cols_cfg)
    subplot_titles_cfg=[d.strftime('%Y-%m-%d') for d in sorted_snap_dates_cfg]
    subplot_titles_cfg.extend([''] * (rows_cfg * cols_cfg - len(subplot_titles_cfg)))
    fig_cfg = make_subplots(rows=rows_cfg, cols=cols_cfg, subplot_titles=subplot_titles_cfg, shared_xaxes=False, shared_yaxes=False, vertical_spacing=0.08, horizontal_spacing=0.05)
    channel_colors_cfg = {"macro": "rgba(60, 60, 60, 0.8)", "meso": "rgba(0, 0, 255, 0.6)", "micro": "rgba(255, 0, 255, 0.7)"}
    channel_styles_cfg = {"macro": "solid", "meso": "dash", "micro": "dot"}

    # Boucle de plotting interne (utilise la logique de zoom de la Cell 30 modifiée)
    for i_cfg, snap_date_cfg_plot in enumerate(sorted_snap_dates_cfg):
         row_idx_cfg = (i_cfg // cols_cfg) + 1; col_idx_cfg = (i_cfg % cols_cfg) + 1
         result_cfg = valid_snapshots_cfg[snap_date_cfg_plot]
         snap_pivots_list_cfg = result_cfg.get("pivots"); snap_channels_cfg = result_cfg.get("channels")

         # Calcul zoom X/Y (Priorité Meso/Micro - identique à Cell 30 modifiée)
         zoom_pivots_data_cfg = []; yaxis_range_subplot_cfg = None; xaxis_range_subplot_cfg = None; zoom_level_used_cfg = "Fallback"
         if snap_channels_cfg:
             for scale in ["meso", "micro"]: # Essai Meso/Micro
                 scale_def = snap_channels_cfg.get(scale, {}); res_p12 = scale_def.get("resistance", ({},{})); sup_p12 = scale_def.get("support", ({},{}))
                 for p_info in [res_p12[0], res_p12[1], sup_p12[0], sup_p12[1]]:
                      if isinstance(p_info, dict) and not pd.isna(p_info.get('time')) and not pd.isna(p_info.get('price')):
                           time_val = pd.to_datetime(p_info['time']).to_pydatetime();
                           if getattr(time_val, 'tzinfo', None) is not None: time_val = time_val.replace(tzinfo=None)
                           zoom_pivots_data_cfg.append({'time': time_val, 'price': p_info['price']})
             if len(zoom_pivots_data_cfg) >= 2: zoom_level_used_cfg = "Meso/Micro"
             else: # Essai Macro
                  zoom_pivots_data_cfg = []
                  scale_def = snap_channels_cfg.get("macro", {}); res_p12 = scale_def.get("resistance", ({},{})); sup_p12 = scale_def.get("support", ({},{}))
                  for p_info in [res_p12[0], res_p12[1], sup_p12[0], sup_p12[1]]:
                      if isinstance(p_info, dict) and not pd.isna(p_info.get('time')) and not pd.isna(p_info.get('price')):
                           time_val = pd.to_datetime(p_info['time']).to_pydatetime();
                           if getattr(time_val, 'tzinfo', None) is not None: time_val = time_val.replace(tzinfo=None)
                           zoom_pivots_data_cfg.append({'time': time_val, 'price': p_info['price']})
                  if len(zoom_pivots_data_cfg) >= 2: zoom_level_used_cfg = "Macro"
         # Calcul ranges si pivots trouvés
         if len(zoom_pivots_data_cfg) >= 2:
              zoom_pivots_df_cfg = pd.DataFrame(zoom_pivots_data_cfg)
              min_p_zoom_cfg = zoom_pivots_df_cfg['price'].min(); max_p_zoom_cfg = zoom_pivots_df_cfg['price'].max()
              min_t_zoom_cfg = zoom_pivots_df_cfg['time'].min(); max_t_zoom_cfg = zoom_pivots_df_cfg['time'].max()
              height_zoom_cfg = max_p_zoom_cfg - min_p_zoom_cfg; margin_y_cfg = max(height_zoom_cfg * 0.30, (min_p_zoom_cfg + max_p_zoom_cfg)/2 * 0.05)
              yaxis_range_subplot_cfg = [min_p_zoom_cfg - margin_y_cfg, max_p_zoom_cfg + margin_y_cfg]
              min_t_zoom_ts_cfg = pd.Timestamp(min_t_zoom_cfg, tz=data_tz); max_t_zoom_ts_cfg = pd.Timestamp(max_t_zoom_cfg, tz=data_tz)
              duration_zoom_cfg = max_t_zoom_ts_cfg - min_t_zoom_ts_cfg if max_t_zoom_ts_cfg > min_t_zoom_ts_cfg else pd.Timedelta(days=1)
              margin_t_before_cfg = max(duration_zoom_cfg * 0.5, pd.Timedelta(days=45)); margin_t_after_cfg = max(duration_zoom_cfg * 0.2, pd.Timedelta(days=20))
              xaxis_range_subplot_cfg = [min_t_zoom_ts_cfg - margin_t_before_cfg, max_t_zoom_ts_cfg + margin_t_after_cfg]
              xaxis_range_subplot_cfg[1] = min(xaxis_range_subplot_cfg[1], pd.Timestamp(snap_date_cfg_plot + pd.Timedelta(days=5), tz=data_tz))
         # Fallback temporel si aucun pivot
         if xaxis_range_subplot_cfg is None:
              zoom_level_used_cfg = "Fallback Time"; lookback_plot_days_cfg = 90
              xaxis_range_subplot_cfg = [pd.Timestamp(snap_date_cfg_plot - pd.Timedelta(days=lookback_plot_days_cfg), tz=data_tz), pd.Timestamp(snap_date_cfg_plot + pd.Timedelta(days=5), tz=data_tz)]
              bars_subplot_fallback_cfg = bars_df[(bars_df['time'] >= xaxis_range_subplot_cfg[0]) & (bars_df['time'] <= snap_date_cfg_plot)].copy()
              if not bars_subplot_fallback_cfg.empty:
                   min_p_cfg = bars_subplot_fallback_cfg['low'].min(); max_p_cfg = bars_subplot_fallback_cfg['high'].max(); margin_y_cfg = (max_p_cfg - min_p_cfg) * 0.1 if (max_p_cfg - min_p_cfg) > 1e-6 else max_p_cfg * 0.05
                   yaxis_range_subplot_cfg = [min_p_cfg - margin_y_cfg, max_p_cfg + margin_y_cfg]

         # Filtrage données & pivots (identique à Cell 30 modifiée)
         # ... (coller ici la partie filtrage de la cellule 30 modifiée, avec suffixe _cfg) ...
         bars_subplot_cfg = bars_df[(bars_df['time'] >= xaxis_range_subplot_cfg[0]) & (bars_df['time'] <= snap_date_cfg_plot)].copy()
         pivots_in_view_df_cfg = pd.DataFrame()
         if snap_pivots_list_cfg:
             pivots_plot_df_cfg = pd.DataFrame(snap_pivots_list_cfg, columns=['time','price','type']); pivots_plot_df_cfg['time'] = pd.to_datetime(pivots_plot_df_cfg['time'])
             pivot_tz_cfg = pivots_plot_df_cfg['time'].dt.tz # Gérer TZ...
             if data_tz is not None and pivot_tz_cfg is None:
                  try: pivots_plot_df_cfg['time'] = pivots_plot_df_cfg['time'].dt.tz_localize(data_tz, ambiguous='infer', nonexistent='shift_forward')
                  except: pass
             elif data_tz is None and pivot_tz_cfg is not None: pivots_plot_df_cfg['time'] = pivots_plot_df_cfg['time'].dt.tz_convert(None)
             elif data_tz is not None and pivot_tz_cfg is not None and hasattr(data_tz, 'zone') and hasattr(pivot_tz_cfg, 'zone') and data_tz.zone != pivot_tz_cfg.zone: pivots_plot_df_cfg['time'] = pivots_plot_df_cfg['time'].dt.tz_convert(data_tz)
             pivots_in_view_df_cfg = pivots_plot_df_cfg[(pivots_plot_df_cfg['time'] >= xaxis_range_subplot_cfg[0]) & (pivots_plot_df_cfg['time'] <= snap_date_cfg_plot)]
         if yaxis_range_subplot_cfg is None: # Fallback Y
              all_prices_in_view_cfg = pd.concat([bars_subplot_cfg['low'], bars_subplot_cfg['high'], pivots_in_view_df_cfg['price']]).dropna()
              if not all_prices_in_view_cfg.empty:
                   min_p_cfg = all_prices_in_view_cfg.min(); max_p_cfg = all_prices_in_view_cfg.max(); margin_y_cfg = (max_p_cfg - min_p_cfg) * 0.1 if (max_p_cfg - min_p_cfg) > 1e-6 else max_p_cfg * 0.05
                   yaxis_range_subplot_cfg = [min_p_cfg - margin_y_cfg, max_p_cfg + margin_y_cfg]

         # Tracer Prix, Pivots, Canaux (identique à Cell 30 modifiée)
         # ... (coller ici la partie tracé de la cellule 30 modifiée, avec suffixe _cfg) ...
         # Plot Prix
         if not bars_subplot_cfg.empty: fig_cfg.add_trace(go.Scatter(x=bars_subplot_cfg['time'], y=bars_subplot_cfg['close'], mode='lines', name='Close', line=dict(color='rgba(150, 150, 150, 0.5)'), showlegend=(i_cfg==0)), row=row_idx_cfg, col=col_idx_cfg)
         # Plot Pivots
         if not pivots_in_view_df_cfg.empty:
              high_x_cfg = pivots_in_view_df_cfg[pivots_in_view_df_cfg['type'] < 0]['time']; high_y_cfg = pivots_in_view_df_cfg[pivots_in_view_df_cfg['type'] < 0]['price']
              low_x_cfg  = pivots_in_view_df_cfg[pivots_in_view_df_cfg['type'] > 0]['time']; low_y_cfg  = pivots_in_view_df_cfg[pivots_in_view_df_cfg['type'] > 0]['price']
              fig_cfg.add_trace(go.Scatter(x=high_x_cfg, y=high_y_cfg, mode='markers', name='High Pivot', marker=dict(color='red', size=5, symbol='diamond-open'), showlegend=(i_cfg==0)), row=row_idx_cfg, col=col_idx_cfg)
              fig_cfg.add_trace(go.Scatter(x=low_x_cfg, y=low_y_cfg, mode='markers', name='Low Pivot', marker=dict(color='green', size=5, symbol='circle-open'), showlegend=(i_cfg==0)), row=row_idx_cfg, col=col_idx_cfg)
         # Plot Canaux
         if snap_channels_cfg and not bars_subplot_cfg.empty:
             plot_start_time_eff_cfg = bars_subplot_cfg['time'].min(); plot_end_time_eff_cfg = snap_date_cfg_plot
             plot_start_time_num_eff_cfg = plot_start_time_eff_cfg.timestamp(); plot_end_time_num_eff_cfg = plot_end_time_eff_cfg.timestamp()
             for channel_name_cfg in ["macro", "meso", "micro"]: # ... (reste identique) ...
                 definition_cfg = snap_channels_cfg.get(channel_name_cfg, {})
                 for line_type_cfg in ["resistance", "support"]:
                     p1_info_cfg, p2_info_cfg = definition_cfg.get(line_type_cfg, ({}, {}))
                     if (isinstance(p1_info_cfg, dict) and isinstance(p2_info_cfg, dict) and # Check validité
                         not pd.isna(p1_info_cfg.get('time')) and not pd.isna(p2_info_cfg.get('time')) and
                         not pd.isna(p1_info_cfg.get('time_numeric')) and not pd.isna(p2_info_cfg.get('time_numeric')) and
                         not pd.isna(p1_info_cfg.get('price')) and not pd.isna(p2_info_cfg.get('price'))):
                         p1_time_num_cfg = p1_info_cfg['time_numeric']; p2_time_num_cfg = p2_info_cfg['time_numeric']; p1_price_cfg = p1_info_cfg['price']; p2_price_cfg = p2_info_cfg['price']; p1_time_cfg = pd.to_datetime(p1_info_cfg['time']); p2_time_cfg = pd.to_datetime(p2_info_cfg['time'])
                         try: # ... (m, c, lignes, markers identiques) ...
                             m_cfg, c_cfg = get_line_params_time(p1_time_num_cfg, p1_price_cfg, p2_time_num_cfg, p2_price_cfg)
                             if m_cfg != np.inf:
                                 line_plot_start_time_dt_cfg = max(p1_time_cfg, plot_start_time_eff_cfg); line_plot_start_time_num_cfg = line_plot_start_time_dt_cfg.timestamp(); line_plot_end_time_dt_cfg = plot_end_time_eff_cfg
                                 line_start_y_plot_cfg = m_cfg * line_plot_start_time_num_cfg + c_cfg; line_end_y_plot_cfg = m_cfg * plot_end_time_num_eff_cfg + c_cfg
                                 fig_cfg.add_trace(go.Scatter(x=[line_plot_start_time_dt_cfg, line_plot_end_time_dt_cfg], y=[line_start_y_plot_cfg, line_end_y_plot_cfg], mode='lines', name=f"{channel_name_cfg.capitalize()} {line_type_cfg.capitalize()}", line=dict(color=channel_colors_cfg[channel_name_cfg], width=1.5, dash=channel_styles_cfg[channel_name_cfg]), legendgroup=f"{channel_name_cfg}_{line_type_cfg}", showlegend=(i_cfg==0)), row=row_idx_cfg, col=col_idx_cfg)
                                 pivots_x_to_mark_cfg = []; pivots_y_to_mark_cfg = []
                                 if xaxis_range_subplot_cfg and p1_time_cfg >= xaxis_range_subplot_cfg[0] and p1_time_cfg <= xaxis_range_subplot_cfg[1]: pivots_x_to_mark_cfg.append(p1_time_cfg); pivots_y_to_mark_cfg.append(p1_price_cfg)
                                 if xaxis_range_subplot_cfg and p2_time_cfg >= xaxis_range_subplot_cfg[0] and p2_time_cfg <= xaxis_range_subplot_cfg[1]: pivots_x_to_mark_cfg.append(p2_time_cfg); pivots_y_to_mark_cfg.append(p2_price_cfg)
                                 if pivots_x_to_mark_cfg: fig_cfg.add_trace(go.Scatter(x=pivots_x_to_mark_cfg, y=pivots_y_to_mark_cfg, mode='markers', marker=dict(color=channel_colors_cfg[channel_name_cfg], size=7, symbol='star' if line_type_cfg == 'resistance' else 'star-open'), showlegend=False, hoverinfo='skip'), row=row_idx_cfg, col=col_idx_cfg)
                         except Exception as e_plotline_cfg: pass # print(f"WARN: Plot line {config_label} {snap_date_cfg_plot.date()}: {e_plotline_cfg}")


         # Appliquer zooms et masquer ticks (identique à Cell 30 modifiée)
         if yaxis_range_subplot_cfg: fig_cfg.update_yaxes(range=yaxis_range_subplot_cfg, row=row_idx_cfg, col=col_idx_cfg)
         if xaxis_range_subplot_cfg: fig_cfg.update_xaxes(range=xaxis_range_subplot_cfg, row=row_idx_cfg, col=col_idx_cfg)
         if row_idx_cfg < rows_cfg: fig_cfg.update_xaxes(showticklabels=False, row=row_idx_cfg, col=col_idx_cfg)
         if col_idx_cfg > 1: fig_cfg.update_yaxes(showticklabels=False, row=row_idx_cfg, col=col_idx_cfg)

    # Layout Final pour ce graphique
    fig_cfg.update_layout(title=f"Snapshots pour {config_label} (Index {config_idx}) - Zoom Meso/Micro",
                          height=250 * rows_cfg + 80, width=1200, hovermode='x unified',
                          legend=dict(orientation="h", yanchor="bottom", y=-0.08, xanchor="center", x=0.5))
    for i_annot_cfg, annot_cfg in enumerate(fig_cfg.layout.annotations):
         if i_annot_cfg < n_snapshots_to_plot_cfg: annot_cfg.update(font=dict(size=9))
    fig_cfg.show()
    # --------------------------------------------------------

print("\n--- Visualisation des snapshots pour les configurations sélectionnées terminée ---")


## 8. Optimisation par Algorithme Génétique (GA)

Cette section utilise un algorithme génétique (implémenté avec la bibliothèque DEAP) pour explorer l'espace des paramètres de stratégies de trading basées sur les canaux multi-échelles pré-calculés. L'objectif est d'identifier des combinaisons de règles et de paramètres qui maximisent une métrique de performance (ici, l'équité finale) sur la période de simulation.

**Étapes Clés :**

1.  **Définition de l'Espace Paramétrique :** Spécifier les différentes options pour chaque paramètre de la stratégie (niveau de canal, type de signal, filtre de tendance, gestion du risque, SL/TP, etc.).
2.  **Création d'Individus (Chromosomes) :** Définir une fonction qui transforme une combinaison de paramètres (un individu) en une structure de stratégie complète (règles, actions).
3.  **Fonction de Fitness :** Implémenter une fonction (`evaluate_strategy`) qui simule une stratégie donnée sur l'historique des prix et des canaux (calculé avec `final_config`) et retourne une valeur de fitness (équité finale).
4.  **Configuration DEAP :** Mettre en place les opérateurs génétiques (sélection, croisement, mutation) et les outils DEAP pour gérer la population et l'évolution.
5.  **Exécution de l'AG :** Lancer le processus d'optimisation sur plusieurs générations.
6.  **Analyse des Résultats :** Visualiser la convergence de la fitness, identifier la meilleure stratégie trouvée et re-simuler sa performance en détail.
7.  **(Optionnel) Re-simulation Manuelle :** Permettre de tester manuellement une configuration spécifique découverte lors d'une exécution précédente ou définie par l'utilisateur.

**Important :** L'historique des canaux utilisé pour l'évaluation de la fitness dans le GA est basé sur la `final_config` sélectionnée dans la section précédente. La performance du GA dépend donc directement de la pertinence de cette configuration de canaux fixe.

### 8.1. Préparation de l'Environnement GA

Cette sous-section configure les éléments nécessaires au fonctionnement de l'algorithme génétique.

*   **Espace Paramétrique (`param_space`) :** Définit les choix possibles pour chaque hyperparamètre de la stratégie de trading (ex: quel canal utiliser, type de signal, paramètres de SL/TP, etc.).
*   **Création de Stratégie (`create_strategy_from_params`) :** Une fonction qui prend une combinaison de paramètres (un "individu" ou "chromosome" pour le GA) et la traduit en un ensemble de règles de trading structurées.
*   **Fonctions Helpers (`get_channel_value_at_time`, `check_conditions`, `calculate_position_size`, `apply_fees`) :** Fonctions utilitaires pour la simulation (calcul de valeur de canal, vérification des conditions de trade, calcul de taille de position, application des frais). La fonction `apply_fees` tente d'utiliser le modèle de frais de QuantConnect ou un fallback simple.
*   **Calcul de l'Historique des Canaux Fixes (`channel_history_df`) :** **Étape cruciale**. Recalcule l'état des canaux Macro/Meso/Micro (en utilisant la `final_config` sélectionnée précédemment) à intervalles réguliers sur toute la période de simulation. Ce DataFrame historique sera utilisé par la fonction de fitness pour déterminer l'état des canaux à chaque pas de temps sans avoir à les recalculer pour chaque individu du GA, optimisant ainsi considérablement le processus.
*   **Configuration DEAP :** Initialise les outils de la bibliothèque DEAP, définissant comment les individus sont créés, évalués (via `evaluate_strategy`), croisés (`mate`), mutés (`mutate`), et sélectionnés (`select`) pour les générations suivantes.

In [ ]:
# CELLULE 7.1.1 : Espace Paramétrique pour le GA

import itertools
import random # Import random ici si pas déjà fait globalement

print("Définition de l'espace paramétrique des stratégies pour le GA...")

# --- Options pour chaque paramètre de la stratégie ---
# (Assurez-vous que ces options correspondent à ce que vous voulez optimiser)
param_space = {
    'trade_level': ['micro', 'meso'],
    'signal_type': ['bounce', 'breakout', 'both'],
    'trend_filter_level': ['none', 'meso', 'macro'],
    'risk_per_trade_pct': [0.0075, 0.01, 0.015, 0.02],
    'min_channel_width_pct': [0.004, 0.008, 0.012, 0.016],
    'bounce_sl_type': ['pct_entry', 'pct_level'],
    'bounce_sl_value': [0.005, 0.01, 0.015, 0.02],
    'bounce_tp_type': ['rr_ratio'], # Type de TP fixe pour l'instant
    'bounce_tp_value': [1.5, 2.0, 2.5, 3.0],
    'bounce_entry_offset': [0.001, 0.002, 0.003],
    'breakout_sl_type': ['pct_level'], # Type de SL fixe pour breakout
    'breakout_sl_value': [0.005, 0.01, 0.015, 0.02],
    'breakout_tp_type': ['rr_ratio'], # Type de TP fixe
    'breakout_tp_value': [1.5, 2.0, 2.5, 3.0],
}

# Obtenir la liste ordonnée des clés (important pour DEAP)
param_keys = list(param_space.keys())

print(f"Espace paramétrique défini avec {len(param_keys)} paramètres.")

# Calculer le nombre total de combinaisons (pour info, peut être très grand)
# total_combinations_possible = 1
# for key in param_keys:
#     total_combinations_possible *= len(param_space[key])
# print(f"Nombre total de combinaisons possibles dans cet espace : {total_combinations_possible:,}")

In [ ]:
# CELLULE NOUVELLE : Fonction pour Créer une Stratégie (Chromosome) depuis les Paramètres

print("Définition de la fonction de création de stratégie...")

def create_strategy_from_params(param_combination, param_keys):
    """
    Crée un dictionnaire de stratégie (chromosome) à partir d'une combinaison de paramètres.
    """
    params = dict(zip(param_keys, param_combination))
    strategy_name = f"L{params['trade_level']}_S{params['signal_type']}_TF{params['trend_filter_level']}" \
                    f"_R{params['risk_per_trade_pct']*100:.0f}_SLB{params['bounce_sl_value']*1000:.0f}" \
                    f"_SLK{params['breakout_sl_value']*1000:.0f}_TP{params['bounce_tp_value']}"

    rules = []

    # --- Actions Modèles (basées sur les params actuels) ---
    current_bounce_long = {
        "action_type": "bounce_long", "order_type": "market",
        f"stop_loss_{params['bounce_sl_type'].replace('_', '_pct_')}" : params['bounce_sl_value'],
        "target_rr_ratio": params['bounce_tp_value']
    }
    current_bounce_short = {
        "action_type": "bounce_short", "order_type": "market",
         f"stop_loss_{params['bounce_sl_type'].replace('pct_entry', 'pct_above_entry').replace('pct_level', 'pct_above_level')}" : params['bounce_sl_value'],
        "target_rr_ratio": params['bounce_tp_value']
    }
    current_breakout_long = {
        "action_type": "breakout_long", "order_type": "market",
        f"stop_loss_{params['breakout_sl_type'].replace('_', '_pct_')}" : params['breakout_sl_value'],
        "target_rr_ratio": params['breakout_tp_value']
    }
    current_breakout_short = {
        "action_type": "breakout_short", "order_type": "market",
        f"stop_loss_{params['breakout_sl_type'].replace('pct_level', 'pct_above_level')}" : params['breakout_sl_value'],
        "target_rr_ratio": params['breakout_tp_value']
    }

    # --- Génération des Règles ---
    trade_level = params['trade_level']
    min_width = params['min_channel_width_pct']
    bounce_offset = params['bounce_entry_offset']
    trend_filter = params['trend_filter_level']

    # Conditions communes pour le filtre de tendance
    trend_cond_long = {}
    trend_cond_short = {}
    if trend_filter != 'none':
        trend_cond_long = {"channel_trend": {"level": trend_filter, "direction": "up"}}
        trend_cond_short = {"channel_trend": {"level": trend_filter, "direction": "down"}}

    # Ajouter règles BOUNCE si signal_type est 'bounce' or 'both'
    if params['signal_type'] in ['bounce', 'both']:
        # Bounce Long Rule
        conditions_bl = {
            "in_position": "none",
            "price_near_level": {"level": f"{trade_level}_support", "threshold_pct": bounce_offset},
            "channel_width_ok": {"level": trade_level, "min_pct": min_width},
            **trend_cond_long # Ajoute le filtre de tendance s'il est défini
        }
        rules.append({"conditions": conditions_bl, "action": current_bounce_long})

        # Bounce Short Rule
        conditions_bs = {
            "in_position": "none",
            "price_near_level": {"level": f"{trade_level}_resistance", "threshold_pct": bounce_offset},
            "channel_width_ok": {"level": trade_level, "min_pct": min_width},
             **trend_cond_short
        }
        rules.append({"conditions": conditions_bs, "action": current_bounce_short})

    # Ajouter règles BREAKOUT si signal_type est 'breakout' or 'both'
    if params['signal_type'] in ['breakout', 'both']:
         # Breakout Long Rule
        conditions_kl = {
            "in_position": "none",
            "price_closes_above": {"level": f"{trade_level}_resistance"},
             # "channel_width_ok": {"level": trade_level, "min_pct": min_width}, # Largeur peut être moins pertinente pour breakout pur
             **trend_cond_long # Filtre optionnel
        }
        rules.append({"conditions": conditions_kl, "action": current_breakout_long})

         # Breakout Short Rule
        conditions_ks = {
            "in_position": "none",
            "price_closes_below": {"level": f"{trade_level}_support"},
             # "channel_width_ok": {"level": trade_level, "min_pct": min_width},
             **trend_cond_short # Filtre optionnel
        }
        rules.append({"conditions": conditions_ks, "action": current_breakout_short})

    # Règle par défaut
    rules.append({ "conditions": {}, "action": "do_nothing" })

    # Assembler le chromosome final
    strategy_chromosome = {
        "name": strategy_name,
        "parameters": params, # Garder une trace des paramètres utilisés
        "general_params": {
            "max_concurrent_trades": 1,
            "risk_per_trade_pct_equity": params['risk_per_trade_pct'],
            "min_channel_width_pct_for_bounce": min_width if params['signal_type'] in ['bounce', 'both'] else 999, # Inactif si pas de bounce
        },
        "rules": rules
    }
    return strategy_chromosome

print("Fonction de création de stratégie définie.")


In [ ]:
# CELLULE 7.1.3 : Définition des Fonctions Helper pour Simulation GA (Syntaxe Finale Corrigée v9)

import pandas as pd
import numpy as np
from datetime import timezone, datetime # Assurer imports
import traceback # Pour le debug si besoin

# --- Imports QC ou Placeholders ---
try:
    from AlgorithmImports import OrderFeeParameters, OrderType, Security, Symbol, ConstantFeeModel, MarketOrder
    print("Imports QC OK pour helpers GA.")
    qcimports_ok_helper = True
except ImportError:
    print("WARN: QC Imports manquants pour helpers GA. Définition de placeholders.")
    qcimports_ok_helper = False
    # --- Placeholders Globaux (Syntaxe Finale Corrigée) ---
    if 'MarketOrder' not in locals():
        class MarketOrder:                              # Correct: Classe sur sa ligne
            def __init__(self, symbol, quantity, time): # Correct: Init indenté
                self.Symbol = symbol
                self.Quantity = quantity
                self.Time = time
                self.Price = 1 # Dummy
                self.Status = 0 # Dummy

    if 'OrderFeeValue' not in locals():             # Check pour la classe externe
        class OrderFeeValue: Amount = 0.0           # Si elle manque, la définir

    if 'OrderFeeParameters' not in locals():
        class OrderFeeParameters:                       # Correct: Classe sur sa ligne
            def __init__(self, security, order):      # Correct: Init indenté
                pass
            class OrderFeeValueInternal_7_1_3: Amount = 0.0 # Nom unique
            Value = OrderFeeValueInternal_7_1_3()         # Correct: Instance comme attribut de classe

    if 'OrderType' not in locals():
        class OrderType:                                # Correct: Classe sur sa ligne
            Market = 1                                  # Correct: Attribut indenté

    if 'Security' not in locals():
        class Security:                                 # Correct: Classe sur sa ligne
            pass                                        # Correct: pass indenté

    if 'Symbol' not in locals():
        class Symbol:                                   # Correct: Classe sur sa ligne
            pass                                        # Correct: pass indenté

    if 'ConstantFeeModel' not in locals():
        class ConstantFeeModel:                         # Correct: Classe sur sa ligne
            def __init__(self, fee):                  # Correct: Init indenté
                self.fee = fee
            def GetOrderFee(self, parameters):        # Correct: Méthode indentée
                # Suppose que OrderFeeParameters (global/placeholder) existe déjà
                if 'OrderFeeParameters' not in globals():
                    raise NameError("Placeholder global OrderFeeParameters non défini")
                # Doit retourner un objet avec .Value.Amount ou un numérique
                return OrderFeeParameters.Value
    # --- Fin Placeholders ---

# --- Définition symbol_security (tentative - Syntaxe Corrigée et Lisibilité Améliorée) ---
print("Définition de l'objet Security pour le symbole...")
if 'symbol_security' not in locals():
    if 'qb' in locals() and 'btc_symbol' in locals() and qcimports_ok_helper:
        try:
            symbol_security = qb.Securities[btc_symbol]
            print(f"Objet Security pour {btc_symbol} récupéré.")
        except Exception as e:
            print(f"WARN: Récupération Security échouée: {e}.")
            qcimports_ok_helper = False # Marquer comme échoué si la récupération rate

    # Vérifier si on a besoin du placeholder (si l'import ou la récupération a échoué, ou si non défini)
    if not qcimports_ok_helper or 'symbol_security' not in locals():
        print("Création placeholder MinimalSecurity...")
        # *** SYNTAXE CORRIGÉE ICI ***
        class MinimalSecurity:                      # Correct: Classe sur sa ligne
            Symbol = 'BTCUSDT'                      # Correct: Attribut indenté
            class QuoteCurrencyClass:               # Correct: Classe imbriquée indentée
                Symbol = 'USDT'
            QuoteCurrency = QuoteCurrencyClass()    # Correct: Instance comme attribut indenté
        symbol_security = MinimalSecurity()         # Correct: Instanciation après définition
else:
    print("Objet 'symbol_security' déjà défini.")

# --- Définition fee_model (tentative - Syntaxe Corrigée et Lisibilité Améliorée) ---
print("Définition du modèle de frais...")
if 'fee_model' not in locals():
    # Tenter de récupérer via qb seulement si les imports sont OK ET symbol_security a QuoteCurrency
    if 'qb' in locals() and qcimports_ok_helper and hasattr(symbol_security, 'QuoteCurrency'):
        try:
            fee_model = qb.BrokerageModel.GetFeeModel(symbol_security)
            print(f"fee_model OK (qb): {type(fee_model)}")
        except Exception as e:
            print(f"WARN: Récupération FeeModel échouée: {e}.")
            qcimports_ok_helper = False # Marquer comme échoué ici aussi pourrait être pertinent

    # Créer placeholder si nécessaire (imports KO, récupération KO, ou fee_model non défini)
    if not qcimports_ok_helper or 'fee_model' not in locals():
        print("Création placeholder ConstantFeeModel(0)...")
        if 'ConstantFeeModel' not in globals(): # Vérifier si la classe placeholder existe
             raise NameError("Placeholder ConstantFeeModel n'a pas été défini globalement")
        fee_model = ConstantFeeModel(0)
else:
    print(f"Modèle de frais 'fee_model' déjà défini: {type(fee_model)}")


print("Définition/Mise à jour des fonctions helper pour la simulation GA...")

# --- Fonctions get_line_params_time, get_channel_value_at_time ---
# (Supposées correctes, mais ajout d'un placeholder pour get_line_params_time si manquant)
if 'get_line_params_time' not in locals():
    print("WARN: Fonction get_line_params_time non trouvée, redéfinition locale.")
    def get_line_params_time(p1tn, p1p, p2tn, p2p):
        diff = p2tn - p1tn
        if abs(diff) < 1e-9:
            return (np.inf, p1tn) # Retourner infini pour pente, et une coordonnée comme 'intercept'
        elif abs(p2p - p1p) < 1e-9:
             return (0.0, p1p) # Pente 0, intercept = prix constant
        else:
            slope = (p2p - p1p) / diff
            intercept = p1p - slope * p1tn
            return (slope, intercept)

def get_channel_value_at_time(channel_info, time_numeric):
    # Vérification initiale plus robuste
    if not isinstance(channel_info, (list, tuple)) or len(channel_info) != 2:
        return np.nan
    if not all(isinstance(p, dict) for p in channel_info):
        return np.nan

    # Utiliser .get avec default pour éviter les erreurs si les clés n'existent pas
    p1 = channel_info[0]
    p2 = channel_info[1]
    p1_time = p1.get('time_numeric', np.nan)
    p1_price = p1.get('price', np.nan)
    p2_time = p2.get('time_numeric', np.nan)
    p2_price = p2.get('price', np.nan)

    # Vérifier si toutes les valeurs nécessaires sont valides
    if any(pd.isna(x) for x in [p1_time, p1_price, p2_time, p2_price]):
        return np.nan

    try:
        slope, intercept = get_line_params_time(p1_time, p1_price, p2_time, p2_price)
        # Si la pente est infinie (ligne verticale), on ne peut pas calculer de valeur à un temps donné
        if slope == np.inf:
            return np.nan
        # Si la pente est nulle, la valeur est constante (l'intercept)
        elif slope == 0.0:
            return intercept
        # Calcul standard
        else:
            return slope * time_numeric + intercept
    except Exception as e:
        # print(f"Erreur dans get_channel_value_at_time: {e}") # Pour debug
        return np.nan

def check_conditions(conditions_dict, current_state):
    if not conditions_dict: return True

    price = current_state.get('price')
    time_numeric = current_state.get('time_numeric')
    position_status_numeric = current_state.get('position_status_numeric') # Changed key
    active_channels = current_state.get('active_channels')

    if any(x is None for x in [price, time_numeric, position_status_numeric, active_channels]):
        return False

    for condition_key, condition_value in conditions_dict.items():
        if condition_key == "in_position":
            is_in_position = (position_status_numeric != 0)
            if is_in_position != condition_value: return False
        elif condition_key == "price_near_level":
            level_id = condition_value.get("level")
            threshold_pct = condition_value.get("threshold_pct", 0.001)
            if not level_id or '_' not in level_id: return False
            slope_name, line_type = level_id.split('_', 1)
            channel_info = active_channels.get(slope_name, {}).get(line_type)
            if not channel_info: return False
            level_value = get_channel_value_at_time(channel_info, time_numeric)
            if pd.isna(level_value): return False
            if not (level_value * (1 - threshold_pct) <= price <= level_value * (1 + threshold_pct)): return False
        elif condition_key == "price_closes_above":
            level_id = condition_value.get("level")
            if not level_id or "_" not in level_id: return False
            slope_name, line_type = level_id.split("_", 1)
            channel_info = active_channels.get(slope_name, {}).get(line_type)
            if not channel_info: return False
            level_value = get_channel_value_at_time(channel_info, time_numeric)
            if pd.isna(level_value) or not (price > level_value): return False
        elif condition_key == "price_closes_below":
            level_id = condition_value.get("level")
            if not level_id or "_" not in level_id: return False
            slope_name, line_type = level_id.split("_", 1)
            channel_info = active_channels.get(slope_name, {}).get(line_type)
            if not channel_info: return False
            level_value = get_channel_value_at_time(channel_info, time_numeric)
            if pd.isna(level_value) or not (price < level_value): return False
        elif condition_key == "channel_width_ok":
            slope_name = condition_value.get("level")
            min_width_pct = condition_value.get("min_pct", 0.01)
            if not slope_name: return False
            resistance_info = active_channels.get(slope_name, {}).get("resistance")
            support_info = active_channels.get(slope_name, {}).get("support")
            if not resistance_info or not support_info: return False
            resistance_value = get_channel_value_at_time(resistance_info, time_numeric)
            support_value = get_channel_value_at_time(support_info, time_numeric)
            if pd.isna(resistance_value) or pd.isna(support_value) or resistance_value <= support_value: return False
            if (resistance_value - support_value) < (price * min_width_pct): return False
        elif condition_key == "channel_trend":
            slope_name = condition_value.get("level")
            required_direction = condition_value.get("direction")
            if not slope_name or not required_direction: return False
            resistance_info = active_channels.get(slope_name, {}).get("resistance")
            support_info = active_channels.get(slope_name, {}).get("support")
            def is_valid_pivot_pair(p_pair):
                 return (isinstance(p_pair, (list, tuple)) and len(p_pair) == 2 and
                         all(isinstance(p, dict) and 'time_numeric' in p and 'price' in p and
                             not pd.isna(p['time_numeric']) and not pd.isna(p['price']) for p in p_pair))
            if not is_valid_pivot_pair(resistance_info) or not is_valid_pivot_pair(support_info): return False
            try:
                mr, _ = get_line_params_time(resistance_info[0]['time_numeric'], resistance_info[0]['price'], resistance_info[1]['time_numeric'], resistance_info[1]['price'])
                ms, _ = get_line_params_time(support_info[0]['time_numeric'], support_info[0]['price'], support_info[1]['time_numeric'], support_info[1]['price'])
                if mr == np.inf or ms == np.inf: return False
                tolerance = 1e-9
                is_trending_up = (mr >= -tolerance and ms >= -tolerance) and not (abs(mr) < tolerance and abs(ms) < tolerance)
                is_trending_down = (mr <= tolerance and ms <= tolerance) and not (abs(mr) < tolerance and abs(ms) < tolerance)
                if required_direction == "up" and not is_trending_up: return False
                if required_direction == "down" and not is_trending_down: return False
            except Exception as e: return False
    return True # All conditions passed

def calculate_position_size(equity, risk_per_trade_pct, entry_price, stop_loss_price):
    if entry_price <= 0 or equity <= 0: return 0.0
    if stop_loss_price is None or pd.isna(stop_loss_price): return 0.0
    cash_at_risk = equity * risk_per_trade_pct
    risk_per_unit = abs(entry_price - stop_loss_price)
    if risk_per_unit < 1e-9: return 0.0
    quantity = cash_at_risk / risk_per_unit
    min_quantity = 1e-5 # Example
    if quantity < min_quantity: return 0.0
    else: return quantity

# --- Fonction apply_fees (Amélioration Lisibilité) ---
def apply_fees(order, symbol_security_obj, fee_model_obj):
    """ Simule l'application des frais en utilisant les objets passés. """
    try:
        # Vérifier si les objets nécessaires et leurs méthodes/attributs existent
        can_use_model = (hasattr(fee_model_obj, 'GetOrderFee') and
                         hasattr(symbol_security_obj, 'Symbol') and
                         hasattr(order, 'Quantity') and
                         hasattr(order, 'Symbol'))

        if can_use_model:
            # Vérifier la présence de la classe nécessaire (importée ou placeholder)
            if 'OrderFeeParameters' not in globals():
                raise NameError("Classe OrderFeeParameters non trouvée globalement.")

            fee_params = OrderFeeParameters(symbol_security_obj, order)
            fee_result = fee_model_obj.GetOrderFee(fee_params)

            if hasattr(fee_result, 'Value') and hasattr(fee_result.Value, 'Amount'):
                fee = fee_result.Value.Amount
            elif isinstance(fee_result, (int, float)):
                fee = fee_result
            else:
                # print(f"WARN: Structure retour GetOrderFee inconnue ({type(fee_result)}). Frais=0.")
                fee = 0.0
            return abs(fee)
        else:
            # Fallback si le modèle ne peut pas être utilisé
            # print("WARN: Utilisation calcul frais fallback (0.1%) dans apply_fees.")
            order_price = getattr(order, 'Price', 1)
            order_price = order_price if order_price is not None and order_price > 0 else 1
            order_quantity = getattr(order, 'Quantity', 0)
            order_value = abs(order_quantity * order_price)
            return order_value * 0.001 # Frais de 0.1%

    except Exception as e:
        print(f"ERREUR dans apply_fees: {e}")
        # traceback.print_exc() # Décommenter pour la trace complète
        return 0.0

print("Fonctions helper pour la simulation GA définies/mises à jour (Syntaxe v9 Finale, Lisibilité Améliorée).")

In [ ]:
# CELLULE 7.1.4 : Calcul de l'Historique des Canaux (basé sur final_config) (DÉBUT MODIFIÉ)

import pandas as pd
from tqdm.notebook import tqdm
import traceback
import numpy as np # Assurer import
import math # Assurer import

print("Calcul de l'historique des canaux basé sur la configuration fixe...")

# --- Vérifications des dépendances ---
if 'bars_df' not in locals(): raise NameError("bars_df non défini.")

# *** AJOUT : Assurer que final_config existe et l'assigner ***
if 'final_config' not in locals():
    raise NameError("La variable 'final_config' (contenant les paramètres de canaux choisis) n'a pas été définie. Exécutez la cellule 'Identification Configuration Cible' (anciennement 16/27) avant celle-ci.")
fixed_channel_config = final_config
# *** FIN AJOUT ***

# Vérifier la fonction de calcul
if 'calculate_all_channels_at_date' not in locals() or not callable(calculate_all_channels_at_date):
    raise NameError("Fonction 'calculate_all_channels_at_date' non définie. Assurez-vous que la cellule précédente (Helper) a été exécutée.")

# --- Paramètres pour ce calcul (Reste identique) ---
lookback_days_hist = 180
zigzag_thresh_hist = 0.05 # Utiliser le seuil standardisé
recalc_frequency_hist = pd.Timedelta(days=1)
hist_start_date = bars_df['time'].min()
hist_end_date = bars_df['time'].max()
data_tz = bars_df['time'].dt.tz

channel_history = {}
last_calculated_channels_hist = None

# --- Génération Dates Recalcul (Reste identique) ---
recalc_dates_hist = pd.date_range(start=hist_start_date + pd.Timedelta(days=lookback_days_hist),
                                end=hist_end_date, freq=recalc_frequency_hist, tz=data_tz)
if recalc_dates_hist.empty:
    raise ValueError("Pas de dates de recalcul générées.")

print(f"Utilisation de la configuration de canaux fixe : '{fixed_channel_config.get('label', 'N/A')}' pour le calcul de l'historique...")

# --- Boucle de Calcul (Reste identique) ---
for recalc_date in tqdm(recalc_dates_hist, desc="Calcul Historique Canaux Fixes"):
    actual_recalc_time = bars_df[bars_df['time'] <= recalc_date]['time'].max()
    if pd.isna(actual_recalc_time) or actual_recalc_time in channel_history: continue

    # Appel à la fonction de calcul (utilise maintenant fixed_channel_config qui est défini)
    pivots, channels = calculate_all_channels_at_date(actual_recalc_time, bars_df, fixed_channel_config, zigzag_threshold=zigzag_thresh_hist)

    if channels is not None:
        channel_history[actual_recalc_time] = channels
        last_calculated_channels_hist = channels
    elif last_calculated_channels_hist is not None:
        channel_history[actual_recalc_time] = last_calculated_channels_hist

if not channel_history:
    raise ValueError("Échec complet du calcul de l'historique des canaux.")

# --- Création DataFrame (Reste identique) ---
channel_history_df = pd.DataFrame.from_dict(channel_history, orient='index')
channel_history_df.index = pd.to_datetime(channel_history_df.index)
channel_history_df = channel_history_df.sort_index()
print(f"Historique des canaux (channel_history_df) créé avec {len(channel_history_df)} entrées.")
# print(channel_history_df.head()) # Décommenter pour vérifier

# --- Préparation loop_bars_all (Reste identique) ---
sim_start_date_ga = pd.Timestamp("2024-01-01", tz=data_tz)
loop_bars_all = bars_df[bars_df['time'] >= sim_start_date_ga].copy()
if loop_bars_all.empty:
    raise ValueError(f"Aucune donnée de barre disponible après {sim_start_date_ga} pour la simulation GA.")
print(f"Période de simulation pour le GA: {loop_bars_all['time'].min()} à {loop_bars_all['time'].max()}")



In [ ]:
# CELLULE GA-1 : Setup de l'Algorithme Génétique avec DEAP (CORRIGÉE v2 + LOGGING)

import random
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
from tqdm.notebook import tqdm
import traceback
import logging

# S'assurer que les imports spécifiques QC sont présents si nécessaire hors de l'env QC
try:
    from AlgorithmImports import MarketOrder, OrderFeeParameters, ConstantFeeModel
except ImportError:
    print("WARN: QuantConnect imports non trouvés. Simulation avec ConstantFeeModel(0).")
    class MarketOrder:
        def __init__(self, symbol, quantity, time): pass
    class OrderFeeParameters:
         def __init__(self, security, order): pass
         class OrderFeeValue:
             Amount = 0.0
         Value = OrderFeeValue()
    class ConstantFeeModel:
        def __init__(self, fee): self.fee = fee
        def GetOrderFee(self, parameters): return OrderFeeParameters.Value


# --- Configuration du Logging ---
log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
# Mettre logging.DEBUG pour voir l'évaluation de chaque individu (très verbeux)
logging.basicConfig(level=logging.INFO, format=log_format)
logger = logging.getLogger("GA_Optimizer")
# ------------------------------

logger.info("Configuration de l'algorithme génétique (DEAP)...")

# --- 1. Vérification des Dépendances ---
if 'strategy_simulation_results' not in locals():
    logger.warning("'strategy_simulation_results' non trouvé.")
if 'calculate_position_size' not in locals(): raise NameError("Fonction 'calculate_position_size' non définie.")
if 'apply_fees' not in locals(): raise NameError("Fonction 'apply_fees' non définie.")
if 'check_conditions' not in locals(): raise NameError("Fonction 'check_conditions' non définie.")
if 'get_channel_value_at_time' not in locals(): raise NameError("Fonction 'get_channel_value_at_time' non définie.")
if 'channel_history_df' not in locals() or channel_history_df.empty:
     raise NameError("L'historique des canaux ('channel_history_df') doit être calculé avant d'exécuter le GA.")
if 'fixed_channel_config' not in locals():
     raise NameError("La configuration fixe des canaux ('fixed_channel_config') doit être définie.")
if 'loop_bars_all' not in locals() or loop_bars_all.empty:
     raise NameError("Le DataFrame 'loop_bars_all' contenant les barres de simulation n'est pas défini ou est vide.")
if 'symbol_security' not in locals(): raise NameError("'symbol_security' non défini.")
if 'fee_model' not in locals():
    logger.warning("Modèle de frais non défini, simulation sans frais.")
    fee_model = ConstantFeeModel(0)
if 'initial_cash' not in locals(): initial_cash = 10000 # Définir une valeur par défaut
if 'btc_symbol' not in locals(): btc_symbol = None # Définir placeholder


# --- 2. Espace Paramétrique (Chromosome) ---
if 'param_space' not in locals(): raise NameError("L'espace paramétrique 'param_space' n'est pas défini.")
if 'param_keys' not in locals(): raise NameError("'param_keys' n'est pas défini.")

# --- 3. Fonction Fitness (Évaluation d'un individu) ---
memoization_cache = {}

def evaluate_strategy(individual):
    """
    Fonction de fitness pour l'algorithme génétique.
    Évalue une stratégie (représentée par 'individual') en utilisant l'historique
    de canaux pré-calculé ('channel_history_df').
    Retourne une tuple (fitness_value,) - DEAP requiert un tuple.
    """
    global initial_cash, loop_bars_all, channel_history_df, btc_symbol, symbol_security, fee_model # Accéder aux variables globales nécessaires

    individual_tuple = tuple(individual)
    # logger.debug(f"Début évaluation individu: {individual_tuple}")
    if individual_tuple in memoization_cache:
        return memoization_cache[individual_tuple]

    try:
        if 'create_strategy_from_params' not in globals(): raise NameError("create_strategy_from_params non définie")
        strategy_config = create_strategy_from_params(individual, param_keys)
        strat_gen_params = strategy_config.get('general_params', {})
        strat_rules = strategy_config.get('rules', [])
        risk_per_trade_pct = strat_gen_params.get('risk_per_trade_pct_equity', 0.01)

        sim_cash = initial_cash; sim_position_qty = 0.0; sim_entry_price = 0.0
        sim_equity = initial_cash; active_stop_loss = None; active_take_profit = None
        last_known_channels_sim_ga = None

        for index, row in loop_bars_all.iterrows():
            current_time = row['time']; current_price = row['close']
            current_high = row['high']; current_low = row['low']
            current_time_num = current_time.timestamp()

            valid_channel_time_index = channel_history_df.index[channel_history_df.index <= current_time]
            current_active_channels = None
            if not valid_channel_time_index.empty:
                valid_channel_time = valid_channel_time_index.max()
                try:
                    current_active_channels = channel_history_df.loc[valid_channel_time].to_dict()
                    last_known_channels_sim_ga = current_active_channels
                except Exception as e: current_active_channels = last_known_channels_sim_ga
            else: current_active_channels = last_known_channels_sim_ga
            if current_active_channels is None: continue

            exit_executed = False
            if sim_position_qty != 0:
                 pnl = 0.0; exit_reason = ""; close_position_now = False; exit_price = current_price
                 if active_stop_loss is not None:
                     if sim_position_qty > 0 and current_low <= active_stop_loss: close_position_now=True; exit_reason="Stop Loss"; exit_price=active_stop_loss
                     elif sim_position_qty < 0 and current_high >= active_stop_loss: close_position_now=True; exit_reason="Stop Loss"; exit_price=active_stop_loss
                 if not close_position_now and active_take_profit is not None:
                      if sim_position_qty > 0 and current_high >= active_take_profit: close_position_now=True; exit_reason="Take Profit"; exit_price=active_take_profit
                      elif sim_position_qty < 0 and current_low <= active_take_profit: close_position_now=True; exit_reason="Take Profit"; exit_price=active_take_profit
                 if close_position_now:
                     exit_order = MarketOrder(btc_symbol, -sim_position_qty, current_time)
                     exit_fee = apply_fees(exit_order, symbol_security, fee_model)
                     pnl = sim_position_qty * (exit_price - sim_entry_price)
                     sim_cash += sim_position_qty * exit_price; sim_cash -= exit_fee
                     sim_position_qty = 0.0; sim_entry_price = 0.0; active_stop_loss = None; active_take_profit = None; exit_executed = True

            if sim_position_qty == 0 and not exit_executed:
                current_state = { 'price': current_price, 'time_numeric': current_time_num, 'position_status': 'none', 'active_channels': current_active_channels }
                for rule in strat_rules:
                    conditions = rule.get('conditions', {}); action_template = rule.get('action')

                    # <-- Logique corrigée pour action_details -->
                    if isinstance(action_template, dict):
                        action_details = action_template
                    elif action_template == "do_nothing":
                        action_details = {"action_type": "do_nothing"}
                    else:
                        action_details = {} # Cas par défaut ou erreur

                    if check_conditions(conditions, current_state):
                        action_type = action_details.get("action_type")
                        if action_type in ["bounce_long", "breakout_long", "bounce_short", "breakout_short"]:
                            entry_price = current_price; signal = 1 if "long" in action_type else -1
                            sl_price = None; tp_price = None; risk_amount_per_unit = 0

                            # --- Calcul SL/TP ---
                            is_bounce_action = 'bounce' in action_type
                            # !! Utilise strategy_config['parameters'] pour les params de l'individu actuel !!
                            strat_origin_params = strategy_config.get('parameters', {})
                            original_sl_type = strat_origin_params.get('bounce_sl_type' if is_bounce_action else 'breakout_sl_type')
                            original_sl_value_key = 'bounce_sl_value' if is_bounce_action else 'breakout_sl_value'
                            sl_val = strat_origin_params.get(original_sl_value_key)

                            if original_sl_type is None or sl_val is None: # Vérifier si les clés existent
                                logger.warning(f"Paramètres SL manquants pour {action_type} dans {strategy_config.get('name')}")
                                continue # Ne pas prendre le trade si SL non défini

                            sl_pct_entry = sl_val if original_sl_type == 'pct_entry' else None
                            sl_pct_level = sl_val if original_sl_type == 'pct_level' else None

                            if signal == 1: # Long SL
                                if sl_pct_entry: sl_price = entry_price * (1 - sl_pct_entry)
                                elif sl_pct_level:
                                    level_name = conditions.get("price_near_level",{}).get("level") or conditions.get("price_closes_above",{}).get("level") or f"{strat_origin_params.get('trade_level','micro')}_resistance" # Ajout fallback niveau
                                    scale, line_type = level_name.split('_')
                                    level_val = get_channel_value_at_time(current_active_channels.get(scale, {}).get(line_type), current_time_num)
                                    sl_price = level_val * (1 - sl_pct_level) if not pd.isna(level_val) else entry_price * (1 - sl_val)
                                else: sl_price = entry_price * (1 - sl_val)
                            else: # Short SL
                                if sl_pct_entry: sl_price = entry_price * (1 + sl_pct_entry)
                                elif sl_pct_level:
                                    level_name = conditions.get("price_near_level",{}).get("level") or conditions.get("price_closes_below",{}).get("level") or f"{strat_origin_params.get('trade_level','micro')}_support" # Ajout fallback niveau
                                    scale, line_type = level_name.split('_')
                                    level_val = get_channel_value_at_time(current_active_channels.get(scale, {}).get(line_type), current_time_num)
                                    sl_price = level_val * (1 + sl_pct_level) if not pd.isna(level_val) else entry_price * (1 + sl_val)
                                else: sl_price = entry_price * (1 + sl_val)

                            target_rr_key = 'bounce_tp_value' if is_bounce_action else 'breakout_tp_value'
                            target_rr = strat_origin_params.get(target_rr_key)

                            if sl_price is not None and target_rr is not None:
                                risk_amount_per_unit = abs(entry_price - sl_price)
                                if risk_amount_per_unit > 1e-9:
                                    tp_price = entry_price + signal * risk_amount_per_unit * target_rr
                            # --- Fin Calcul SL/TP ---

                            if sl_price is not None and risk_amount_per_unit > 1e-9:
                                 sim_equity_before_trade = sim_cash
                                 qty_to_trade = calculate_position_size(sim_equity_before_trade, risk_per_trade_pct, entry_price, sl_price)
                                 if qty_to_trade > 0:
                                     entry_order = MarketOrder(btc_symbol, signal * qty_to_trade, current_time)
                                     entry_fee = apply_fees(entry_order, symbol_security, fee_model)
                                     sim_position_qty = signal * qty_to_trade; sim_entry_price = entry_price
                                     sim_cash -= sim_position_qty * sim_entry_price; sim_cash -= entry_fee
                                     active_stop_loss = sl_price; active_take_profit = tp_price
                                     # Pas de log de trade ici pour la performance
                                     break # Sortir boucle règles
                        elif action_type == "do_nothing":
                            break # Sortir boucle règles

        # --- Fin Logique Trading ---
        sim_equity = sim_cash + sim_position_qty * current_price

        final_equity = sim_cash + sim_position_qty * current_price
        fitness = final_equity if final_equity > 1 else 1.0
        result_tuple = (fitness,)
        memoization_cache[individual_tuple] = result_tuple
        # logger.debug(f"Fin évaluation individu -> Fitness: {fitness:.2f}")
        return result_tuple

    except Exception as e:
        logger.exception(f"Erreur pendant l'évaluation de l'individu {individual_tuple}:")
        memoization_cache[individual_tuple] = (0.0,)
        return (0.0,)


# --- 4. Configuration DEAP (Identique) ---
# (Assurez-vous que cette partie est bien présente et correcte dans votre cellule)
# Si FitnessMax existe déjà, la ligne creator.create peut lever une erreur,
# ce qui est normal si vous exécutez la cellule plusieurs fois.
try:
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
except Exception:
    logger.info("DEAP creators 'FitnessMax' et/ou 'Individual' existent déjà.")
    pass # Continuer si elles existent déjà

toolbox = base.Toolbox()
gene_generators = []
for key in param_keys:
    options = param_space[key]
    if isinstance(options[0], str): gene_generators.append(lambda opts=options: random.choice(opts))
    elif isinstance(options[0], (int, float)):
         if isinstance(options[0], float) or len(options) == 2:
             min_val, max_val = min(options), max(options)
             if isinstance(min_val, float) or isinstance(max_val, float): gene_generators.append(lambda mn=min_val, mx=max_val: random.uniform(mn, mx))
             else: gene_generators.append(lambda opts=options: random.choice(opts))
         else: gene_generators.append(lambda opts=options: random.choice(opts))
    else: raise TypeError(f"Type de paramètre non supporté pour {key}: {type(options[0])}")

toolbox.register("individual", tools.initCycle, creator.Individual, gene_generators, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_strategy)
toolbox.register("mate", tools.cxTwoPoint)
def mutate_individual(individual, indpb=0.1): # Fonction mutate identique
    for i in range(len(individual)):
        if random.random() < indpb:
            key = param_keys[i]; options = param_space[key]
            if isinstance(options[0], str): individual[i] = random.choice(options)
            elif isinstance(options[0], (int, float)):
                 if isinstance(options[0], float) or len(options) == 2:
                     min_val, max_val = min(options), max(options)
                     if isinstance(min_val, float) or isinstance(max_val, float): individual[i] = random.uniform(min_val, max_val)
                     else: individual[i] = random.choice(options)
                 else: individual[i] = random.choice(options)
    return individual,
toolbox.register("mutate", mutate_individual, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

logger.info("Configuration DEAP (corrigée v2 avec logging) terminée.")

### 8.2. Exécution de l'Algorithme Génétique

Cette cellule lance l'algorithme génétique DEAP configuré précédemment.

*   **Paramètres GA :** `POPULATION_SIZE`, `GENERATIONS`, `CXPB` (probabilité de croisement), `MUTPB` (probabilité de mutation) contrôlent le processus d'évolution.
*   **Initialisation :** Une population initiale d'individus (stratégies aléatoires) est créée.
*   **Évolution (`algorithms.eaSimple`) :** La boucle principale du GA est exécutée. À chaque génération :
    *   La fitness de chaque individu est évaluée (simulation de la stratégie).
    *   Les meilleurs individus sont sélectionnés.
    *   Des opérateurs de croisement et de mutation sont appliqués pour créer la nouvelle génération.
*   **Suivi :** Des statistiques (min, max, avg fitness) sont collectées à chaque génération. Le meilleur individu trouvé (`hof`) est conservé.
*   **Résultat :** Affiche la meilleure fitness (équité finale) atteinte et les paramètres de la stratégie correspondante.

In [ ]:
# CELLULE GA-2 : Exécution de l'Algorithme Génétique (Avec Logging)

logger.info("Lancement de l'algorithme génétique...")
memoization_cache.clear() # Vider le cache avant une nouvelle exécution

# Paramètres du GA
POPULATION_SIZE = 50
GENERATIONS = 20
CXPB = 0.7
MUTPB = 0.2

# Initialiser la population
logger.info(f"Initialisation de la population (taille={POPULATION_SIZE})...")
pop = toolbox.population(n=POPULATION_SIZE)
logger.info("Population initialisée.")

# Garder une trace du meilleur individu
hof = tools.HallOfFame(1)

# Statistiques pour suivre l'évolution
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# Lancer l'algorithme
# Note : eaSimple loggue déjà des infos si verbose=True. On ajoute des logs avant/après.
logger.info(f"Début de l'évolution pour {GENERATIONS} générations...")
pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=GENERATIONS,
                                   stats=stats, halloffame=hof, verbose=True) # verbose=True donne les stats DEAP par génération

logger.info("Algorithme génétique terminé.")

if hof: # Vérifier si le HallOfFame n'est pas vide
    best_individual = hof[0]
    best_fitness = best_individual.fitness.values[0]
    logger.info(f"Meilleure Fitness (Equity Finale) trouvée : {best_fitness:,.2f}")
    logger.info("Paramètres du meilleur individu :")
    best_params_dict = dict(zip(param_keys, best_individual))
    for key, value in best_params_dict.items():
        if isinstance(value, float):
            logger.info(f"  - {key}: {value:.4f}")
        else:
            logger.info(f"  - {key}: {value}")
else:
    logger.warning("Aucun individu dans le HallOfFame après l'exécution du GA.")
    best_individual = None # Pour éviter des erreurs plus loin

# Vider le cache après l'exécution
memoization_cache.clear()
logger.debug("Cache de fitness vidé.")

### 8.3. Analyse des Résultats du GA

Après l'exécution de l'algorithme génétique, cette cellule analyse les résultats :

1.  **Visualisation de la Convergence :** Un graphique montre l'évolution de la fitness maximale et moyenne au fil des générations. Cela permet de vérifier si l'algorithme a convergé vers une solution stable.
2.  **Re-simulation de la Meilleure Stratégie :** La stratégie correspondant au meilleur individu trouvé par le GA (stocké dans `hof[0]`) est simulée à nouveau, cette fois en enregistrant plus de détails (courbe d'équité, liste des trades).
3.  **Calcul des Métriques de Performance :** Pour la meilleure stratégie, des métriques clés sont calculées et affichées (Équité Finale, PnL Total, Win Rate, Max Drawdown, Nombre de Trades, Frais Totaux).
4.  **Affichage de la Courbe d'Équité :** La courbe de performance de la meilleure stratégie est tracée.
5.  **Rappel des Paramètres :** Les paramètres exacts de la meilleure stratégie trouvée sont affichés.

In [ ]:
# CELLULE GA-3 : Analyse des Résultats de l'AG (CORRIGÉE - Erreur action_details)

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import heapq
import logging # Assurer l'import si ce n'est pas fait plus haut
import pandas as pd # Assurer l'import
import numpy as np # Assurer l'import

# Récupérer le logger défini précédemment
logger = logging.getLogger("GA_Optimizer")

logger.info("--- Analyse des Résultats de l'Algorithme Génétique ---")

# --- 1. Visualisation de la Convergence (Identique) ---
if 'logbook' in locals() and logbook: # S'assurer que logbook existe
    gen = logbook.select("gen")
    fit_max = logbook.select("max")
    fit_avg = logbook.select("avg")

    fig, ax1 = plt.subplots(figsize=(12, 6))
    color = 'tab:red'; ax1.set_xlabel('Génération'); ax1.set_ylabel('Max Fitness (Equity)', color=color)
    ax1.plot(gen, fit_max, color=color, label="Max Fitness"); ax1.tick_params(axis='y', labelcolor=color)
    ax1.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0f'))
    ax2 = ax1.twinx(); color = 'tab:blue'; ax2.set_ylabel('Avg Fitness (Equity)', color=color)
    ax2.plot(gen, fit_avg, color=color, linestyle='--', label="Avg Fitness"); ax2.tick_params(axis='y', labelcolor=color)
    ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0f'))
    fig.tight_layout(); plt.title('Convergence de la Fitness (Equity) au fil des Générations')
    lines, labels = ax1.get_legend_handles_labels(); lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='center right'); plt.grid(True); plt.show()
else:
    logger.warning("Logbook non disponible, impossible de tracer la convergence.")

# --- 2. Re-simulation de la Meilleure Stratégie Trouvée (Avec Logging) ---
# Assurer que 'best_individual' existe et a été assigné dans la cellule précédente
if 'best_individual' in locals() and best_individual is not None:
    logger.info("Re-simulation de la meilleure stratégie trouvée pour analyse détaillée...")
    # S'assurer que les fonctions nécessaires sont disponibles
    if 'create_strategy_from_params' not in globals(): raise NameError("create_strategy_from_params non définie")
    if 'param_keys' not in globals(): raise NameError("param_keys non définis")

    best_strategy_config = create_strategy_from_params(best_individual, param_keys)
    best_strategy_name = best_strategy_config['name']
    logger.info(f"Stratégie à re-simuler: {best_strategy_name}")

    # (Logique de simulation - identique à evaluate_strategy mais stocke trades/equity)
    sim_cash = initial_cash; sim_position_qty = 0.0; sim_entry_price = 0.0
    sim_equity = initial_cash; active_stop_loss = None; active_take_profit = None
    equity_curve_list_best = [{'time': sim_start_date - pd.Timedelta(hours=1), 'equity': initial_cash}]
    sim_trades_best = []
    strat_rules_best = best_strategy_config.get('rules', [])
    strat_gen_params_best = best_strategy_config.get('general_params', {})
    risk_per_trade_pct_best = strat_gen_params_best.get('risk_per_trade_pct_equity', 0.01)
    last_known_channels_sim_best = None

    # Utiliser loop_bars_all qui doit être défini (contient les barres pour la simulation)
    if 'loop_bars_all' not in locals() or loop_bars_all.empty:
         raise NameError("Le DataFrame 'loop_bars_all' pour la simulation n'est pas défini ou est vide.")

    for index, row in loop_bars_all.iterrows(): # Pas de tqdm ici pour éviter surcharge logs
        current_time = row['time']; current_price = row['close']
        current_high = row['high']; current_low = row['low']
        current_time_num = current_time.timestamp()

        # Trouver canaux actifs
        valid_channel_time_index = channel_history_df.index[channel_history_df.index <= current_time]
        current_active_channels = None
        if not valid_channel_time_index.empty:
            valid_channel_time = valid_channel_time_index.max()
            try:
                current_active_channels = channel_history_df.loc[valid_channel_time].to_dict()
                last_known_channels_sim_best = current_active_channels
            except Exception as e: current_active_channels = last_known_channels_sim_best
        else: current_active_channels = last_known_channels_sim_best
        if current_active_channels is None:
            sim_equity = sim_cash + sim_position_qty * current_price
            equity_curve_list_best.append({'time': current_time, 'equity': sim_equity})
            continue

        # --- Logique de Trading (Sorties puis Entrées) ---
        exit_executed = False
        # 1. Gestion Sorties (SL/TP)
        if sim_position_qty != 0:
            pnl = 0.0; exit_reason = ""; close_position_now = False; exit_price = current_price
            if active_stop_loss is not None:
                if sim_position_qty > 0 and current_low <= active_stop_loss: close_position_now=True; exit_reason="Stop Loss"; exit_price=active_stop_loss
                elif sim_position_qty < 0 and current_high >= active_stop_loss: close_position_now=True; exit_reason="Stop Loss"; exit_price=active_stop_loss
            if not close_position_now and active_take_profit is not None:
                 if sim_position_qty > 0 and current_high >= active_take_profit: close_position_now=True; exit_reason="Take Profit"; exit_price=active_take_profit
                 elif sim_position_qty < 0 and current_low <= active_take_profit: close_position_now=True; exit_reason="Take Profit"; exit_price=active_take_profit
            if close_position_now:
                exit_order = MarketOrder(btc_symbol, -sim_position_qty, current_time)
                exit_fee = apply_fees(exit_order, symbol_security, fee_model)
                pnl = sim_position_qty * (exit_price - sim_entry_price)
                sim_cash += sim_position_qty * exit_price; sim_cash -= exit_fee
                sim_trades_best.append({'time': current_time, 'type': f'exit_{"long" if sim_position_qty > 0 else "short"}', 'price': exit_price, 'size': -sim_position_qty, 'pnl': pnl - exit_fee, 'fee': exit_fee, 'reason': exit_reason})
                sim_position_qty = 0.0; sim_entry_price = 0.0; active_stop_loss = None; active_take_profit = None; exit_executed = True

        # 2. Gestion Entrées
        if sim_position_qty == 0 and not exit_executed:
            current_state = { 'price': current_price, 'time_numeric': current_time_num, 'position_status': 'none', 'active_channels': current_active_channels }
            for rule in strat_rules_best:
                conditions = rule.get('conditions', {}); action_template = rule.get('action')

                # <-- CORRECTION ICI (appliquée dans la boucle de re-simulation)
                if isinstance(action_template, dict):
                    action_details = action_template
                elif action_template == "do_nothing":
                    action_details = {"action_type": "do_nothing"}
                else:
                    action_details = {}

                if check_conditions(conditions, current_state):
                    action_type = action_details.get("action_type")
                    if action_type in ["bounce_long", "breakout_long", "bounce_short", "breakout_short"]:
                        entry_price = current_price; signal = 1 if "long" in action_type else -1
                        sl_price = None; tp_price = None; risk_amount_per_unit = 0

                        # --- Recalcul SL/TP (Identique) ---
                        is_bounce_action = 'bounce' in action_type
                        best_strat_origin_params = best_strategy_config.get('parameters', {})
                        original_sl_type = best_strat_origin_params['bounce_sl_type'] if is_bounce_action else best_strat_origin_params['breakout_sl_type']
                        original_sl_value_key_suffix = 'bounce_sl_value' if is_bounce_action else 'breakout_sl_value'
                        sl_val = best_strat_origin_params[original_sl_value_key_suffix]
                        sl_pct_entry = sl_val if original_sl_type == 'pct_entry' else None
                        sl_pct_level = sl_val if original_sl_type == 'pct_level' else None
                        if signal == 1: # Long SL
                            if sl_pct_entry: sl_price = entry_price * (1 - sl_pct_entry)
                            elif sl_pct_level:
                                level_name = conditions.get("price_near_level",{}).get("level") or conditions.get("price_closes_above",{}).get("level") or f"{best_strat_origin_params['trade_level']}_resistance"
                                scale, line_type = level_name.split('_')
                                level_val = get_channel_value_at_time(current_active_channels.get(scale, {}).get(line_type), current_time_num)
                                sl_price = level_val * (1 - sl_pct_level) if not pd.isna(level_val) else entry_price * (1 - sl_val)
                            else: sl_price = entry_price * (1 - sl_val)
                        else: # Short SL
                            if sl_pct_entry: sl_price = entry_price * (1 + sl_pct_entry)
                            elif sl_pct_level:
                                level_name = conditions.get("price_near_level",{}).get("level") or conditions.get("price_closes_below",{}).get("level") or f"{best_strat_origin_params['trade_level']}_support"
                                scale, line_type = level_name.split('_')
                                level_val = get_channel_value_at_time(current_active_channels.get(scale, {}).get(line_type), current_time_num)
                                sl_price = level_val * (1 + sl_pct_level) if not pd.isna(level_val) else entry_price * (1 + sl_val)
                            else: sl_price = entry_price * (1 + sl_val)
                        target_rr_key_suffix = 'bounce_tp_value' if is_bounce_action else 'breakout_tp_value'
                        target_rr = best_strat_origin_params[target_rr_key_suffix]
                        if sl_price is not None and target_rr is not None:
                            risk_amount_per_unit = abs(entry_price - sl_price)
                            if risk_amount_per_unit > 1e-9:
                                tp_price = entry_price + signal * risk_amount_per_unit * target_rr
                        # --- Fin Recalcul SL/TP ---

                        if sl_price is not None and risk_amount_per_unit > 1e-9:
                             sim_equity_before_trade = sim_cash
                             qty_to_trade = calculate_position_size(sim_equity_before_trade, risk_per_trade_pct_best, entry_price, sl_price)
                             if qty_to_trade > 0:
                                 entry_order = MarketOrder(btc_symbol, signal * qty_to_trade, current_time)
                                 entry_fee = apply_fees(entry_order, symbol_security, fee_model)
                                 sim_position_qty = signal * qty_to_trade; sim_entry_price = entry_price
                                 sim_cash -= sim_position_qty * sim_entry_price; sim_cash -= entry_fee
                                 active_stop_loss = sl_price; active_take_profit = tp_price
                                 # Enregistrer le trade simulé pour analyse
                                 sim_trades_best.append({'time': current_time, 'type': 'buy' if signal > 0 else 'sell', 'price': entry_price, 'size': sim_position_qty, 'fee': entry_fee, 'sl': active_stop_loss, 'tp': active_take_profit, 'reason': action_type})
                                 break # Sortir boucle règles
                    elif action_type == "do_nothing":
                        break # Sortir boucle règles

        # 3. Mise à jour Equity finale
        sim_equity = sim_cash + sim_position_qty * current_price
        equity_curve_list_best.append({'time': current_time, 'equity': sim_equity})

    # --- Fin Boucle Barre par Barre (Re-simulation) ---

    # --- 3. Affichage des Résultats de la Meilleure Stratégie (Identique) ---
    best_equity_curve = pd.DataFrame(equity_curve_list_best).set_index('time')['equity']
    best_trades_df = pd.DataFrame(sim_trades_best)
    best_final_equity = best_equity_curve.iloc[-1] if not best_equity_curve.empty else initial_cash

    logger.info(f"--- Performance Détaillée de la Meilleure Stratégie Trouvée ({best_strategy_name}) ---")
    logger.info(f"Équité Finale: {best_final_equity:,.2f} USDT")
    # (Calcul des métriques et affichage des logs identiques à la version précédente)
    nb_trades_best = len(best_trades_df[best_trades_df['type'].isin(['buy', 'sell'])])
    total_pnl_best = best_trades_df['pnl'].sum() if 'pnl' in best_trades_df.columns else 0.0
    total_fees_best = best_trades_df['fee'].sum() if 'fee' in best_trades_df.columns else 0.0
    winning_trades_best = best_trades_df[best_trades_df['pnl'] > 0]['pnl'].count() if 'pnl' in best_trades_df.columns else 0
    losing_trades_best = best_trades_df[best_trades_df['pnl'] <= 0]['pnl'].count() if 'pnl' in best_trades_df.columns else 0
    total_closed_trades_best = winning_trades_best + losing_trades_best
    win_rate_best = (winning_trades_best / total_closed_trades_best * 100) if total_closed_trades_best > 0 else 0.0
    max_dd_best = 0.0; peak_best = -np.inf
    if not best_equity_curve.empty:
        for equity_value in best_equity_curve:
            if equity_value > peak_best: peak_best = equity_value
            drawdown = (peak_best - equity_value) / peak_best if peak_best > 0 else 0
            if drawdown > max_dd_best: max_dd_best = drawdown

    logger.info(f"Nombre de Trades (Entrées): {nb_trades_best}")
    logger.info(f"Total PnL Net: {total_pnl_best:,.2f} USDT")
    logger.info(f"Total Fees: {total_fees_best:,.2f} USDT")
    logger.info(f"Win Rate: {win_rate_best:.1f}%")
    logger.info(f"Max Drawdown: {max_dd_best*100:.1f}%")

    plt.figure(figsize=(14, 7))
    best_equity_curve.plot(title=f'Courbe d\'Équité - Meilleure Stratégie GA: {best_strategy_name[:60]}...', grid=True)
    plt.ylabel("Valeur Portefeuille (USDT)")
    plt.xlabel("Date")
    plt.show()

    logger.info("Paramètres de la meilleure stratégie trouvée par le GA:")
    best_params_final = best_strategy_config.get('parameters', {})
    for key, value in best_params_final.items():
         if isinstance(value, float): logger.info(f"  - {key}: {value:.4f}")
         else: logger.info(f"  - {key}: {value}")
else:
    logger.error("Aucun meilleur individu trouvé par le GA. Impossible d'analyser les résultats.")

### 8.4. Re-simulation Manuelle d'une Configuration Spécifique

Cette cellule permet de tester ou de vérifier la performance d'une configuration de stratégie spécifique en dehors du processus d'optimisation du GA.

*   **Définition Manuelle des Paramètres :** L'utilisateur définit explicitement un dictionnaire (`params_run1` dans l'exemple) contenant tous les paramètres de la stratégie à simuler.
*   **Création de la Stratégie :** La fonction `create_strategy_from_params` est utilisée pour construire la structure de la stratégie à partir des paramètres manuels.
*   **Simulation :** La même logique de simulation que celle utilisée dans la fonction de fitness du GA est exécutée pour cette stratégie spécifique, en enregistrant la courbe d'équité et les trades.
*   **Analyse et Affichage :** Les métriques de performance et la courbe d'équité pour cette simulation manuelle sont calculées et affichées, permettant une comparaison avec les résultats du GA ou d'autres benchmarks.

In [ ]:
# CELLULE 7.4 : Re-simulation d'une Configuration Spécifique (Syntaxe Placeholder MinimalSecurity CORRIGÉE v8)

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import heapq
import logging
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import traceback

# --- Assurer imports QC ou placeholders (Syntaxe Corrigée pour cette cellule) ---
try:
    from AlgorithmImports import MarketOrder, OrderFeeParameters, ConstantFeeModel, OrderType, Security, Symbol
    print("Imports QuantConnect OK pour re-sim spécifique (v8).")
    _qc_imports_ok_resim = True
except ImportError:
    print("WARN: QC Imports manquants pour re-sim spécifique (v8). Définition de placeholders.")
    _qc_imports_ok_resim = False
    # --- Placeholders (Syntaxe Corrigée avec Indentation) ---
    if 'MarketOrder' not in locals():
        class MarketOrder:
            def __init__(self, symbol, quantity, time):
                self.Symbol = symbol; self.Quantity = quantity; self.Time = time; self.Price = 1; self.Status = 0

    if 'OrderFeeValue' not in locals():
         class OrderFeeValue:
              Amount = 0.0

    if 'OrderFeeParameters' not in locals():
        class OrderFeeParameters:
            def __init__(self, security, order): pass
            class OrderFeeValueInternal: Amount = 0.0 # Nom différent pour éviter conflit
            Value = OrderFeeValueInternal()

    if 'ConstantFeeModel' not in locals():
        class ConstantFeeModel:
            def __init__(self, fee): self.fee = fee
            def GetOrderFee(self, parameters):
                if 'OrderFeeParameters' not in globals(): raise NameError("Placeholder OrderFeeParameters non défini")
                return OrderFeeParameters.Value

    if 'OrderType' not in locals():
        class OrderType:
            Market = 1

    if 'Security' not in locals():
        class Security:
            pass

    if 'Symbol' not in locals():
        class Symbol:
            pass
    # --- Fin Placeholders ImportError ---

# --- Assurer définition des variables globales ---
if 'btc_symbol' not in locals(): btc_symbol = 'BTCUSDT'

# --- Définition/Vérification symbol_security (Syntaxe MinimalSecurity CORRIGÉE) ---
if 'symbol_security' not in locals():
    print("Vérification/Création 'symbol_security' pour re-sim...")
    if 'qb' in locals() and _qc_imports_ok_resim:
        try:
            symbol_security = qb.Securities[btc_symbol]
            print(f"Objet Security pour {btc_symbol} récupéré via qb.")
        except Exception as e_sec:
            print(f"WARN: Impossible de récupérer Security via qb: {e_sec}. Création placeholder.")
            _qc_imports_ok_resim = False # Marquer comme échoué

    # Créer placeholder si import/récupération échoue OU si déjà marqué comme échoué
    if not _qc_imports_ok_resim or 'symbol_security' not in locals():
        print("Création placeholder MinimalSecurity pour re-sim...")
        # *** SYNTAXE CORRIGÉE ICI ***
        class MinimalSecurity:
            Symbol = btc_symbol         # Attribut de classe
            class QCC:                  # Classe interne indentée
                Symbol = 'USDT'         # Attribut de classe interne indenté
            QuoteCurrency = QCC()       # Instanciation indentée
        symbol_security = MinimalSecurity()
        # *** FIN CORRECTION SYNTAXE ***
else:
    print("'symbol_security' déjà défini pour re-sim.")

# --- Définition/Vérification fee_model (Bloc vérifié) ---
if 'fee_model' not in locals():
    print("Vérification/Création 'fee_model' pour re-sim...")
    if 'qb' in locals() and _qc_imports_ok_resim and hasattr(symbol_security, 'QuoteCurrency') and not isinstance(symbol_security, MinimalSecurity):
        try:
            fee_model = qb.BrokerageModel.GetFeeModel(symbol_security)
            print(f"Modèle de frais récupéré via qb: {fee_model.__class__.__name__}")
        except Exception as e_fee:
            print(f"WARN: Impossible de récupérer FeeModel via qb: {e_fee}. Utilisation ConstantFeeModel(0).")
            _qc_imports_ok_resim = False # Marquer comme échoué

    if not _qc_imports_ok_resim or 'fee_model' not in locals():
        print("Création placeholder ConstantFeeModel(0) pour re-sim...")
        if 'ConstantFeeModel' not in locals(): # S'assurer que la classe placeholder est définie
             class ConstantFeeModel:
                 def __init__(self, fee): self.fee = fee
                 def GetOrderFee(self, parameters):
                     if 'OrderFeeParameters' not in globals(): raise NameError("Placeholder OrderFeeParameters non trouvé")
                     return OrderFeeParameters.Value
        fee_model = ConstantFeeModel(0)
else:
     print("'fee_model' déjà défini pour re-sim.")

if 'initial_cash' not in locals(): initial_cash = 10000
# --- Fin Vérifications Globales ---

# Récupérer logger
logger = logging.getLogger("GA_Optimizer_Resim")
if not logger.hasHandlers(): # Configurer si pas déjà fait
    log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    logging.basicConfig(level=logging.INFO, format=log_format, force=True)

logger.info("--- Re-simulation Manuelle d'une Stratégie Spécifique (v8) ---")

# --- Définir les Paramètres de la Stratégie à Re-tester ---
params_run1 = { # (Paramètres Run1 ~19.4k - Vérifiez si ce sont les bons)
    'trade_level': 'meso', 'signal_type': 'breakout', 'trend_filter_level': 'none',
    'risk_per_trade_pct': 0.0199, 'min_channel_width_pct': 0.0062,
    'bounce_sl_type': 'pct_entry', 'bounce_sl_value': 0.0105, 'bounce_tp_type': 'rr_ratio',
    'bounce_tp_value': 2.1194, 'bounce_entry_offset': 0.0015,
    'breakout_sl_type': 'pct_level', 'breakout_sl_value': 0.0120,
    'breakout_tp_type': 'rr_ratio', 'breakout_tp_value': 2.9670
}
if 'param_keys' not in locals(): raise NameError("Variable 'param_keys' non définie.")
# S'assurer que param_keys correspond bien aux clés de params_run1
missing_keys_in_run1 = [k for k in param_keys if k not in params_run1]
if missing_keys_in_run1: raise ValueError(f"Clés manquantes: {missing_keys_in_run1}")
param_combination_run1 = [params_run1.get(key) for key in param_keys]

# --- Vérifier les fonctions nécessaires ---
required_funcs = ['create_strategy_from_params', 'check_conditions', 'get_channel_value_at_time', 'calculate_position_size', 'apply_fees', 'get_line_params_time']
for func_name in required_funcs:
    if func_name not in globals() and func_name not in locals(): raise NameError(f"Fonction '{func_name}' non définie.")
if 'loop_bars_all' not in locals() or loop_bars_all.empty: raise NameError("'loop_bars_all' requis.")
if 'channel_history_df' not in locals() or channel_history_df.empty: raise NameError("'channel_history_df' requis.")

# --- Créer et Simuler la Stratégie ---
# (Utilise la version de create_strategy_from_params qui prend param_keys globalement)
if 'create_strategy_from_params' not in globals(): raise NameError("create_strategy_from_params non définie")
strategy_config_run1 = create_strategy_from_params(param_combination_run1)
strategy_name_run1 = strategy_config_run1['name']
logger.info(f"Re-simulation de la configuration spécifique : {strategy_name_run1}")

# --- Initialisation et Boucle de Simulation (Identique aux cellules précédentes) ---
sim_cash_run1 = initial_cash; sim_position_qty_run1 = 0.0; sim_entry_price_run1 = 0.0
sim_equity_run1 = initial_cash; active_stop_loss_run1 = None; active_take_profit_run1 = None
sim_start_time_run1 = loop_bars_all['time'].iloc[0] if not loop_bars_all.empty else pd.Timestamp.now(tz='UTC')
equity_curve_list_run1 = [{'time': sim_start_time_run1 - pd.Timedelta(hours=1), 'equity': initial_cash}]
sim_trades_run1 = []
strat_rules_run1 = strategy_config_run1.get('rules', [])
risk_per_trade_pct_run1 = strategy_config_run1['parameters']['risk_per_trade_pct']
origin_params_run1 = strategy_config_run1.get('parameters', {})
last_known_channels_sim_run1 = None
position_status_numeric_run1 = 0 # Ajouter statut numérique

for index, row in tqdm(loop_bars_all.iterrows(), total=len(loop_bars_all), desc=f"Simulating {strategy_name_run1[:20]}...", leave=False):
    # --- Début Boucle Simulation (Copier/Coller depuis Analyse GA) ---
    current_time=row['time']; current_price=row['close']; current_high=row['high']; current_low=row['low']; current_time_num=current_time.timestamp()
    current_active_channels = None;
    valid_channel_time_index = channel_history_df.index[channel_history_df.index <= current_time]
    if not valid_channel_time_index.empty:
        valid_channel_time = valid_channel_time_index.max();
        try: current_active_channels = channel_history_df.loc[valid_channel_time].to_dict(); last_known_channels_sim_run1=current_active_channels;
        except: current_active_channels=last_known_channels_sim_run1;
    else: current_active_channels=last_known_channels_sim_run1;
    if current_active_channels is None:
         sim_equity_run1 = sim_cash_run1 + sim_position_qty_run1 * current_price; equity_curve_list_run1.append({'time': current_time, 'equity': sim_equity_run1}); continue;

    exit_executed=False;
    if position_status_numeric_run1 != 0: # Sorties
        pnl=0.; exit_reason=''; close=False; exit_p=current_price; sl=active_stop_loss_run1; tp=active_take_profit_run1; qty=sim_position_qty_run1; ep=sim_entry_price_run1;
        if sl is not None:
            if qty>0 and current_low<=sl: close=True; exit_reason="SL"; exit_p=sl;
            elif qty<0 and current_high>=sl: close=True; exit_reason="SL"; exit_p=sl;
        if not close and tp is not None:
            if qty>0 and current_high>=tp: close=True; exit_reason="TP"; exit_p=tp;
            elif qty<0 and current_low<=tp: close=True; exit_reason="TP"; exit_p=tp;
        if close:
            # S'assurer que MarketOrder est défini (placeholder ou réel)
            if 'MarketOrder' not in globals(): raise NameError("MarketOrder non défini")
            order=MarketOrder(btc_symbol,-qty,current_time); fee=apply_fees(order,symbol_security,fee_model); pnl=qty*(exit_p-ep); sim_cash_run1+=qty*exit_p-fee;
            sim_trades_run1.append({'time':current_time,'type':f'exit_{"long" if qty>0 else "short"}','price':exit_p,'size':-qty,'pnl':pnl-fee,'fee':fee,'reason':exit_reason,'equity':sim_cash_run1});
            sim_position_qty_run1=0.; sim_entry_price_run1=0.; active_stop_loss_run1=None; active_take_profit_run1=None; exit_executed=True; position_status_numeric_run1=0;

    if position_status_numeric_run1 == 0 and not exit_executed: # Entrées
        cs={'price':current_price,'time_numeric':current_time_num,'position_status_numeric':position_status_numeric_run1,'active_channels':current_active_channels};
        for rule in strat_rules_run1:
            cond=rule.get('conditions',{}); act_tmpl=rule.get('action');
            if isinstance(act_tmpl,dict): ad=act_tmpl;
            elif act_tmpl=="do_nothing": ad={"action_type":"do_nothing"};
            else: ad={};
            if check_conditions(cond, cs):
                aty=ad.get("action_type");
                if aty in ["bounce_long","breakout_long","bounce_short","breakout_short"]:
                    entry_p=current_price; sig=1 if "long" in aty else -1; slp=None; tpp=None; rpu=0;
                    is_b='bounce' in aty;
                    sltk='bounce_sl_type' if is_b else 'breakout_sl_type'; slvk='bounce_sl_value' if is_b else 'breakout_sl_value'; tpvk='bounce_tp_value' if is_b else 'breakout_tp_value';
                    oslt=origin_params_run1.get(sltk); slv=origin_params_run1.get(slvk); trr=origin_params_run1.get(tpvk);
                    if oslt is None or slv is None or trr is None: continue;
                    # Logique SL/TP
                    if sig==1:
                        if oslt=='pct_entry': slp=entry_p*(1-slv);
                        elif oslt=='pct_level': ln_cond=cond.get("price_near_level",{}).get("level") or cond.get("price_closes_above",{}).get("level"); ln=ln_cond or f"{origin_params_run1['trade_level']}_resistance"; s,lt=ln.split('_'); lv=get_channel_value_at_time(current_active_channels.get(s,{}).get(lt),current_time_num); slp=lv*(1-slv) if not pd.isna(lv) else entry_p*(1-slv);
                        else: slp=entry_p*(1-slv);
                    else:
                        if oslt=='pct_entry': slp=entry_p*(1+slv);
                        elif oslt=='pct_level': ln_cond=cond.get("price_near_level",{}).get("level") or cond.get("price_closes_below",{}).get("level"); ln=ln_cond or f"{origin_params_run1['trade_level']}_support"; s,lt=ln.split('_'); lv=get_channel_value_at_time(current_active_channels.get(s,{}).get(lt),current_time_num); slp=lv*(1+slv) if not pd.isna(lv) else entry_p*(1+slv);
                        else: slp=entry_p*(1+slv);
                    if slp is not None: rpu=abs(entry_p-slp);
                    if rpu>1e-9: tpp=entry_p+sig*rpu*trr;
                    if slp is not None and rpu>1e-9:
                        eq_b4=sim_cash_run1; qty=calculate_position_size(eq_b4,risk_per_trade_pct_run1,entry_p,slp);
                        if qty>0:
                             # S'assurer que MarketOrder est défini
                             if 'MarketOrder' not in globals(): raise NameError("MarketOrder non défini")
                             order=MarketOrder(btc_symbol,sig*qty,current_time); fee=apply_fees(order,symbol_security,fee_model);
                             sim_position_qty_run1=sig*qty; sim_entry_price_run1=entry_p; sim_cash_run1-=sim_position_qty_run1*sim_entry_price_run1+fee;
                             active_stop_loss_run1=slp; active_take_profit_run1=tpp; position_status_numeric_run1=sig;
                             sim_trades_run1.append({'time':current_time,'type':'buy' if sig>0 else 'sell','price':entry_p,'size':sim_position_qty_run1,'fee':fee,'sl':slp,'tp':tpp,'reason':aty,'equity':sim_cash_run1});
                             break;
                elif aty=="do_nothing": break;
    # --- Fin Boucle Simulation ---

    sim_equity_run1 = sim_cash_run1 + sim_position_qty_run1 * current_price
    equity_curve_list_run1.append({'time': current_time, 'equity': sim_equity_run1})
# --- Fin Boucle Barre par Barre ---


# --- Analyse et Affichage Résultats pour Run1 ---
equity_curve_run1 = pd.DataFrame(equity_curve_list_run1).set_index('time')['equity']
trades_run1_df = pd.DataFrame(sim_trades_run1)
final_equity_run1 = equity_curve_run1.iloc[-1] if not equity_curve_run1.empty else initial_cash
logger.info(f"--- Performance Détaillée Stratégie Spécifique ({strategy_name_run1}) ---")
logger.info(f"Équité Finale: {final_equity_run1:,.2f} USDT")
if not trades_run1_df.empty:
    nb_trades_run1 = len(trades_run1_df[trades_run1_df['type'].isin(['buy', 'sell'])])
    exit_trades_run1 = trades_run1_df[trades_run1_df['type'].str.startswith('exit')] # Correct: Utiliser les exits pour PnL
    total_pnl_run1 = exit_trades_run1['pnl'].sum() if 'pnl' in exit_trades_run1.columns and not exit_trades_run1.empty else 0.0
    total_fees_run1 = trades_run1_df['fee'].sum() if 'fee' in trades_run1_df.columns else 0.0
    winning_trades_run1 = exit_trades_run1[exit_trades_run1['pnl'] > 0]['pnl'].count() if 'pnl' in exit_trades_run1.columns else 0
    losing_trades_run1 = exit_trades_run1[exit_trades_run1['pnl'] <= 0]['pnl'].count() if 'pnl' in exit_trades_run1.columns else 0
    total_closed_trades_run1 = len(exit_trades_run1) # Utiliser le nombre de sorties
    win_rate_run1 = (winning_trades_run1 / total_closed_trades_run1 * 100) if total_closed_trades_run1 > 0 else 0.0
else: nb_trades_run1=0; total_pnl_run1=0.0; total_fees_run1=0.0; win_rate_run1=0.0; logger.info("Aucun trade exécuté.")
max_dd_run1 = 0.0;
if not equity_curve_run1.empty:
    rolling_max_run1 = equity_curve_run1.cummax(); daily_drawdown_run1 = equity_curve_run1 / rolling_max_run1 - 1.0
    max_dd_run1 = abs(daily_drawdown_run1.min()) if not daily_drawdown_run1.empty else 0.0
logger.info(f"Nb Trades (Entrées): {nb_trades_run1}")
logger.info(f"Total PnL Net (Sorties): {total_pnl_run1:,.2f} USDT")
logger.info(f"Total Fees: {total_fees_run1:,.2f} USDT")
logger.info(f"Win Rate (Sorties): {win_rate_run1:.1f}%")
logger.info(f"Max Drawdown: {max_dd_run1*100:.1f}%")

# --- Plotting ---
plt.figure(figsize=(14, 7))
plot_title_run1 = f"Courbe d'Équité - Re-Sim Spécifique: {strategy_name_run1[:60]}..."
if not equity_curve_run1.empty:
    equity_curve_run1.plot(title=plot_title_run1, grid=True)
    # Ajouter les marqueurs de trades
    if not trades_run1_df.empty:
        entries = trades_run1_df[trades_run1_df['type'].isin(['buy','sell'])]
        exits = trades_run1_df[trades_run1_df['type'].str.startswith('exit')]
        # S'assurer que les index de temps existent dans la courbe d'équité avant de plotter
        valid_entry_times = entries.index.intersection(equity_curve_run1.index)
        valid_exit_times = exits.index.intersection(equity_curve_run1.index)
        if not entries.loc[valid_entry_times].empty:
            plt.scatter(entries.loc[valid_entry_times].index, equity_curve_run1.loc[valid_entry_times], marker='^', color='lime', s=60, label='Entrée', zorder=5)
        if not exits.loc[valid_exit_times].empty:
             plt.scatter(exits.loc[valid_exit_times].index, equity_curve_run1.loc[valid_exit_times], marker='v', color='red', s=60, label='Sortie', zorder=5)
        if not entries.empty or not exits.empty:
             plt.legend()
else:
    plt.title(f"{plot_title_run1} (Pas de données d'équité)")

plt.ylabel("Valeur Portefeuille (USDT)"); plt.xlabel("Date"); plt.show()

logger.info("Paramètres de cette stratégie spécifique:")
for key, value in origin_params_run1.items():
     if isinstance(value, float): logger.info(f"  - {key}: {value:.4f}")
     else: logger.info(f"  - {key}: {value}")